# PLOTTER OF FILL PARAMETERS

In [1]:
# -- enabling TOC for this notebook
#!jupyter nbextension enable toc2/main
# How to install extensions for everything:
# pip install --user jupyter_contrib_nbextensions
# jupyter contrib nbextension install --user
# jupyter nbextension enable toc2/main

import pandas as pd
import numpy as np
import numpy.ma as ma
import pickle
import time
from operator import add
from scipy.constants import c as clight
from scipy.optimize import curve_fit
import gzip
import tarfile
from glob import glob
from collections import OrderedDict
import seaborn as sns
import matplotlib.dates as mdates
from dateutil import tz
import itertools
from scipy.integrate import cumtrapz
%matplotlib inline
#pl.style.library['seaborn-colorblind']
#pl.style.library['classic']
sns.set_style("whitegrid")

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
### 
print 'loading Nikos\' helperFunctions!'
%run helperFunctions.ipynb

loading Nikos' helperFunctions!
Version 0.03. This is the latest version.
Please help me to improve it reporting bugs to guido.sterbini@cern.ch.
Your platform is Linux-3.10.0-693.2.2.el7.x86_64-x86_64-with-redhat-6.9-Carbon
Your folder is /eos/user/n/nkarast/lumimod
Your IP is 172.17.0.12
2017-12-13 18:22:52


## Overall Parameters

In [8]:
#----------------------------------------------------------------------
print 'Running the BunchList IPy notebook... Have you updated the list?'
%run BunchList.ipynb
#----------------------------------------------------------------------


#----------------------------------------------------------------------
input_directory =  '/eos/user/l/lumimod/2017/SB_analysis/fill_<FILLNUMBER>/'
resc                    = ''
SB_filename             = "fill_<FILLNUMBER><RESC>.pkl.gz"
SB_fits_filename        = "fill_<FILLNUMBER>_fits<RESC>.pkl.gz"
SB_model_filename       = "fill_<FILLNUMBER>_sbmodel<RESC>.pkl.gz"
SB_burnoff_filename     = "fill_<FILLNUMBER>_sigma_burnoff<RESC>.pkl.gz"
Cycle_filename          = "fill_<FILLNUMBER>_cycle<RESC>.pkl.gz"
Cycle_model_filename    = "fill_<FILLNUMBER>_cycle_model<RESC>.pkl.gz"
Lumi_filename           = "fill_<FILLNUMBER>_lumi_calc<RESC>.pkl.gz"
Massi_filename          = 'fill_<FILLNUMBER>_lumi_meas.pkl.gz'
#----------------------------------------------------------------------

#----------------------------------------------------------------------
fills_bmodes_file       = '/eos/user/l/lumimod/2017/SB_analysis/fills_and_bmodes.pkl'
#----------------------------------------------------------------------

Running the BunchList IPy notebook... Have you updated the list?


---


## DEFINE FILL PARAMETERS:

____

In [10]:
# ----------------------


filln = 6432


# ----------------------


my_bunches = None
for key in bunches_dict.keys():
    if filln >= key[0] and filln < key[1]:
        my_bunches = bunches_dict[key]


# ----------------------

In [11]:
cycle_filename       = input_directory.replace("<FILLNUMBER>", str(filln))+Cycle_filename.replace("<FILLNUMBER>", str(filln)).replace("<RESC>", str(resc))
cycle_filename       = input_directory.replace("<FILLNUMBER>", str(filln))+Cycle_filename.replace("<FILLNUMBER>", str(filln)).replace("<RESC>", str(resc)) 
sb_filename          = input_directory.replace("<FILLNUMBER>", str(filln))+SB_filename.replace("<FILLNUMBER>", str(filln)).replace("<RESC>", str(resc))
lumi_filename        = input_directory.replace("<FILLNUMBER>", str(filln))+Lumi_filename.replace("<FILLNUMBER>", str(filln)).replace("<RESC>", str(resc))
massi_filename       = input_directory.replace("<FILLNUMBER>", str(filln))+Massi_filename.replace("<FILLNUMBER>", str(filln))
cycleModel_filename  = input_directory.replace("<FILLNUMBER>", str(filln))+Cycle_model_filename.replace("<FILLNUMBER>", str(filln)).replace("<RESC>", str(resc))
sbfits_filename      = input_directory.replace("<FILLNUMBER>", str(filln))+SB_fits_filename.replace("<FILLNUMBER>", str(filln)).replace("<RESC>", str(resc))
lifetime_filename    = sb_filename.replace('.pkl.gz', '_lifetime.pkl.gz')

In [12]:
# Load the necessary files:
print "#Plotter : Loading files for fill {}".format(filln)
try:
    with gzip.open(cycle_filename, 'rb') as fid:
        filln_CycleDict           = pickle.load(fid)
except:
    print "#Plotter : Issues loading Cycle Data"

try:
    with gzip.open(sb_filename, 'rb') as fid:
        filln_StableBeamsDict           = pickle.load(fid)
except:
    print "#Plotter : Issues loading SB Data"

try:
    with gzip.open(lumi_filename, 'rb') as fid:
        filln_LumiCalcDict           = pickle.load(fid)
except:
    print "#Plotter : Issues loading Lumi Calc Data"

try:
    with gzip.open(massi_filename, 'rb') as fid:
        filln_LumiMeasDict           = pickle.load(fid)
except:
    print "#Plotter : Issues loading Lumi Measured Data"

try:
    with gzip.open(lifetime_filename, 'rb') as fid:
        filln_LifetimeDict           = pickle.load(fid)
except:
    print "#Plotter : Issues loading Lifetime Data"

try:
    with gzip.open(cycleModel_filename, 'rb') as fid:
        filln_CycleModelDict           = pickle.load(fid)
except:
    print "#Plotter : Issues loading Cycle Model Data"

try:
    with gzip.open(sbfits_filename, 'rb') as fid:
        filln_SBFitsDict           = pickle.load(fid)
except:
    print "#Plotter : Issues loading SB Fits Data"


# BMODES
bmodes, filln_list = getBmodesDF(fills_bmodes_file)

#Plotter : Loading files for fill 6432


_____

## CYCLE PLOTS
---

### Cycle Emittances

In [ ]:
# ------------ OPTIONS -----------------
show_only = 'injection' #show_only == 'injection', 'startRamp', 'endRamp', 'startSB'
showAll = True
if show_only is not None:
    showAll = False


%matplotlib notebook
%matplotlib notebook

#---------------------------------------
dict_intervals_two_beams = filln_CycleDict
t_start_fill, t_end_fill, t_fill_len, t_ref = getCycleDataTimes(bmodes, filln)

fig_bbbemit = pl.figure("Emittances B1", figsize=(15, 7))
fig_bbbemit.set_facecolor('w')
ax_b1_h = pl.subplot(2,1,1)

if showAll or show_only == 'injection':
    ax_b1_h.plot(getFilledSlotsArray(dict_intervals_two_beams, "beam_1", "Injection",       "at_start",  mask_invalid=True), ma.masked_invalid(np.array(dict_intervals_two_beams['beam_1']['Injection']['at_start']['emith'])), '.', color='blue',   markersize=8, label='Injected')
if showAll or show_only == 'startRamp':
    ax_b1_h.plot(getFilledSlotsArray(dict_intervals_two_beams, "beam_1", "Injection",       "at_end",    mask_invalid=True), ma.masked_invalid(np.array(dict_intervals_two_beams['beam_1']['Injection']['at_end']['emith'])),   '.', color='orange', markersize=8, label='Start Ramp')
if showAll or show_only == 'endRamp':
    ax_b1_h.plot(getFilledSlotsArray(dict_intervals_two_beams, "beam_1", "he_before_SB",    "at_start",  mask_invalid=True), ma.masked_invalid(np.array(dict_intervals_two_beams['beam_1']['he_before_SB']['at_start']['emith'])),  '.', color='green',  markersize=8, label='End Ramp')
if showAll or show_only == 'startSB':
    ax_b1_h.plot(getFilledSlotsArray(dict_intervals_two_beams, "beam_1", "he_before_SB",    "at_end",    mask_invalid=True), ma.masked_invalid(np.array(dict_intervals_two_beams['beam_1']['he_before_SB']['at_end']['emith'])),    '.', color='red',    markersize=8, label='Start SB')
ax_b1_h.set_ylabel("B1 $\mathbf{\epsilon_{H}}$ [$\mathbf{\mu}$m]", fontsize=14, fontweight='bold')
ax_b1_h.minorticks_on()
ax_b1_h.set_ylim(1,3)

ax_b1_h.text(0.5, 0.9, "Injected: {:.2f}$\pm${:.2f} | Start Ramp: {:.2f}$\pm${:.2f} | End Ramp: {:.2f}$\pm${:.2f} | Start SB: {:.2f}$\pm${:.2f}".format(
            np.nanmean(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_1']['Injection']['at_start']['emith']))), 
            np.nanstd(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_1']['Injection']['at_start']['emith']))),
            np.nanmean(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_1']['Injection']['at_end']['emith']))), 
            np.nanstd(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_1']['Injection']['at_end']['emith']))),
            np.nanmean(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_1']['he_before_SB']['at_start']['emith']))), 
            np.nanstd(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_1']['he_before_SB']['at_start']['emith']))),
            np.nanmean(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_1']['he_before_SB']['at_end']['emith']))), 
            np.nanstd(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_1']['he_before_SB']['at_end']['emith'])))         ), 
            horizontalalignment='center', verticalalignment='top', transform=ax_b1_h.transAxes, 
            bbox=dict(facecolor='white', edgecolor='gray', boxstyle='round,pad=0.5', alpha=0.7), fontweight='bold', fontsize=9)

## Shrink current axis by 20%
box = ax_b1_h.get_position()
ax_b1_h.set_position([box.x0, box.y0, box.width*0.8, box.height])
# Put a legend to the right of the current axis
ax_b1_h.legend(loc='center left', bbox_to_anchor=(1, 0.5), fontsize=12, numpoints=1)
ax_b1_h.grid('on', which='both')
        


ax_b1_v = pl.subplot(2,1,2, sharex=ax_b1_h, sharey=ax_b1_h)
if showAll or show_only == 'injection':
    ax_b1_v.plot(getFilledSlotsArray(dict_intervals_two_beams, "beam_1", "Injection",     "at_start",  mask_invalid=True), ma.masked_invalid(np.array(dict_intervals_two_beams['beam_1']['Injection']['at_start']['emitv'])), '.', color='blue',   markersize=8, label='Injected')
if showAll or show_only == 'startRamp':		
    ax_b1_v.plot(getFilledSlotsArray(dict_intervals_two_beams, "beam_1", "Injection",     "at_end",    mask_invalid=True), ma.masked_invalid(np.array(dict_intervals_two_beams['beam_1']['Injection']['at_end']['emitv'])),   '.', color='orange', markersize=8, label='Start Ramp')
if showAll or show_only == 'endRamp':
    ax_b1_v.plot(getFilledSlotsArray(dict_intervals_two_beams, "beam_1", "he_before_SB",  "at_start",  mask_invalid=True), ma.masked_invalid(np.array(dict_intervals_two_beams['beam_1']['he_before_SB']['at_start']['emitv'])),  '.', color='green',  markersize=8, label='End Ramp')
if showAll or show_only == 'startSB':
    ax_b1_v.plot(getFilledSlotsArray(dict_intervals_two_beams, "beam_1", "he_before_SB",  "at_end",    mask_invalid=True), ma.masked_invalid(np.array(dict_intervals_two_beams['beam_1']['he_before_SB']['at_end']['emitv'])),    '.', color='red',    markersize=8, label='Start SB')
ax_b1_v.set_ylabel("B1 $\mathbf{\epsilon_{V}}$ [$\mathbf{\mu}$m]", fontsize=14, fontweight='bold')
ax_b1_v.minorticks_on()
ax_b1_v.set_ylim(1,3)

ax_b1_v.set_xlabel("Bunch Slots [25ns]", fontsize=14, fontweight='bold')
ax_b1_v.text(0.5, 0.9, "Injected: {:.2f}$\pm${:.2f} | Start Ramp: {:.2f}$\pm${:.2f} | End Ramp: {:.2f}$\pm${:.2f} | Start SB: {:.2f}$\pm${:.2f}".format(
            np.nanmean(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_1']['Injection']['at_start']['emitv']))), 
            np.nanstd(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_1']['Injection']['at_start']['emitv']))),
            np.nanmean(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_1']['Injection']['at_end']['emitv']))),  
            np.nanstd(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_1']['Injection']['at_end']['emitv']))),
            np.nanmean(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_1']['he_before_SB']['at_start']['emitv']))), 
            np.nanstd(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_1']['he_before_SB']['at_start']['emitv']))),
            np.nanmean(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_1']['he_before_SB']['at_end']['emitv']))),   
            np.nanstd(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_1']['he_before_SB']['at_end']['emitv']))             )), 
            horizontalalignment='center', verticalalignment='top', transform=ax_b1_v.transAxes,
            bbox=dict(facecolor='white', edgecolor='gray', boxstyle='round,pad=0.5', alpha=0.7), fontweight='bold', fontsize=9)

## Shrink current axis by 20%
box = ax_b1_v.get_position()
ax_b1_v.set_position([box.x0, box.y0, box.width * 0.8, box.height])
# Put a legend to the right of the current axis
ax_b1_v.legend(loc='center left', bbox_to_anchor=(1, 0.5), fontsize=12, numpoints=1)
ax_b1_v.grid('on', which='both')

# - - * - - * - - * - - * - - * - - * - - * - - * - - * - - * - - * - - * - - * - - * - - * - - * - - * - - * - - * - - * - - * - - * - - * - - * - - * - - * - -
# beam 2  - H
fig_bbbemit2 = pl.figure("Emittances B2", figsize=(15, 7))
fig_bbbemit2.set_facecolor('w')
ax_b2_h = pl.subplot(2,1,1)

if showAll or show_only == 'injection':
    ax_b2_h.plot(getFilledSlotsArray(dict_intervals_two_beams, "beam_2", "Injection",     "at_start",  mask_invalid=True), ma.masked_invalid(np.array(dict_intervals_two_beams['beam_2']['Injection']['at_start']['emith'])), '.', color='blue',   markersize=8, label='Injected')
if showAll or show_only == 'startRamp':		
    ax_b2_h.plot(getFilledSlotsArray(dict_intervals_two_beams, "beam_2", "Injection",     "at_end",    mask_invalid=True), ma.masked_invalid(np.array(dict_intervals_two_beams['beam_2']['Injection']['at_end']['emith'])),   '.', color='orange', markersize=8, label='Start Ramp')
if showAll or show_only == 'endRamp':
    ax_b2_h.plot(getFilledSlotsArray(dict_intervals_two_beams, "beam_2", "he_before_SB",  "at_start",  mask_invalid=True), ma.masked_invalid(np.array(dict_intervals_two_beams['beam_2']['he_before_SB']['at_start']['emith'])),  '.', color='green',  markersize=8, label='End Ramp')
if showAll or show_only == 'startSB':
    ax_b2_h.plot(getFilledSlotsArray(dict_intervals_two_beams, "beam_2", "he_before_SB",  "at_end",     mask_invalid=True), ma.masked_invalid(np.array(dict_intervals_two_beams['beam_2']['he_before_SB']['at_end']['emith'])),    '.', color='red',    markersize=8, label='Start SB')
ax_b2_h.set_ylabel("B2 $\mathbf{\epsilon_{H}}$ [$\mathbf{\mu}$m]", fontsize=14, fontweight='bold')
ax_b2_h.minorticks_on()
ax_b2_h.set_ylim(1,3)
ax_b2_h.text(0.5, 0.9, "Injected: {:.2f}$\pm${:.2f} | Start Ramp: {:.2f}$\pm${:.2f} | End Ramp: {:.2f}$\pm${:.2f} | Start SB: {:.2f}$\pm${:.2f}".format(
            np.nanmean(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_2']['Injection']['at_start']['emith']))),
            np.nanstd(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_2']['Injection']['at_start']['emith']))),
            np.nanmean(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_2']['Injection']['at_end']['emith']))),  
            np.nanstd(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_2']['Injection']['at_end']['emith']))),
            np.nanmean(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_2']['he_before_SB']['at_start']['emith']))), 
            np.nanstd(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_2']['he_before_SB']['at_start']['emith']))),                                                                                                                                             
            np.nanmean(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_2']['he_before_SB']['at_end']['emith']))),   
            np.nanstd(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_2']['he_before_SB']['at_end']['emith'])))     ),  
            horizontalalignment='center', verticalalignment='top', transform=ax_b2_h.transAxes,
            bbox=dict(facecolor='white', edgecolor='gray', boxstyle='round,pad=0.5', alpha=0.7), fontweight='bold', fontsize=9)
## Shrink current axis by 20%
box = ax_b2_h.get_position()
ax_b2_h.set_position([box.x0, box.y0, box.width * 0.8, box.height])
# Put a legend to the right of the current axis
ax_b2_h.legend(loc='center left', bbox_to_anchor=(1, 0.5), fontsize=12, numpoints=1)
ax_b2_h.grid('on', which='both')

# - - * - - * - - * - - * - - * - - * - - * - - * - - * - - * - - * - - * - - * - - * - - * - - * - - * - - * - - * - - * - - * - - * - - * - - * - - * - - * - -
# beam 2  - V
ax_b2_v = pl.subplot(2,1,2, sharex=ax_b2_h, sharey=ax_b2_h)

if showAll or show_only == 'injection':
    ax_b2_v.plot(getFilledSlotsArray(dict_intervals_two_beams, "beam_2", "Injection",     "at_start",  mask_invalid=True), ma.masked_invalid(np.array(dict_intervals_two_beams['beam_2']['Injection']['at_start']['emitv'])), '.', color='blue',   markersize=8, label='Injected')
if showAll or show_only == 'startRamp':		
    ax_b2_v.plot(getFilledSlotsArray(dict_intervals_two_beams, "beam_2", "Injection",     "at_end",    mask_invalid=True), ma.masked_invalid(np.array(dict_intervals_two_beams['beam_2']['Injection']['at_end']['emitv'])),   '.', color='orange', markersize=8, label='Start Ramp')
if showAll or show_only == 'endRamp':
    ax_b2_v.plot(getFilledSlotsArray(dict_intervals_two_beams, "beam_2", "he_before_SB",  "at_start",  mask_invalid=True), ma.masked_invalid(np.array(dict_intervals_two_beams['beam_2']['he_before_SB']['at_start']['emitv'])),  '.', color='green',  markersize=8, label='End Ramp')
if showAll or show_only == 'startSB':
    ax_b2_v.plot(getFilledSlotsArray(dict_intervals_two_beams, "beam_2", "he_before_SB",  "at_end",    mask_invalid=True), ma.masked_invalid(np.array(dict_intervals_two_beams['beam_2']['he_before_SB']['at_end']['emitv'])),    '.', color='red',    markersize=8, label='Start SB')
ax_b2_v.set_ylabel("B2 $\mathbf{\epsilon_{V}}$ [$\mathbf{\mu}$m]", fontsize=14, fontweight='bold')
ax_b2_v.set_xlabel("Bunch Slots [25ns]", fontsize=14, fontweight='bold')
ax_b2_v.minorticks_on()
ax_b2_v.set_ylim(1,3)

ax_b2_v.text(0.5, 0.9, "Injected: {:.2f}$\pm${:.2f} | Start Ramp: {:.2f}$\pm${:.2f} | End Ramp: {:.2f}$\pm${:.2f} | Start SB: {:.2f}$\pm${:.2f}".format(
            np.nanmean(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_2']['Injection']['at_start']['emitv']))), 
            np.nanstd(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_2']['Injection']['at_start']['emitv']))),
            np.nanmean(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_2']['Injection']['at_end']['emitv']))),  
            np.nanstd(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_2']['Injection']['at_end']['emitv']))),
            np.nanmean(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_2']['he_before_SB']['at_start']['emitv']))), 
            np.nanstd(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_2']['he_before_SB']['at_start']['emitv']))),
            np.nanmean(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_2']['he_before_SB']['at_end']['emitv']))),   
            np.nanstd(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_2']['he_before_SB']['at_end']['emitv'])))       ), 
            horizontalalignment='center', verticalalignment='top', transform=ax_b2_v.transAxes,
            bbox=dict(facecolor='white', edgecolor='gray', boxstyle='round,pad=0.5', alpha=0.7), fontweight='bold', fontsize=9)
## Shrink current axis by 20%
box = ax_b2_v.get_position()
ax_b2_v.set_position([box.x0, box.y0, box.width * 0.8, box.height])
# Put a legend to the right of the current axis
ax_b2_v.legend(loc='center left', bbox_to_anchor=(1, 0.5), fontsize=12, numpoints=1)
ax_b2_v.grid('on', which='both')

# tref string
tref_string = datetime.fromtimestamp(t_ref)
subtitle    = 'Fill {} : Started on {}'.format(filln, tref_string)

fig_bbbemit.suptitle(subtitle, fontsize=16, fontweight='bold')
fig_bbbemit2.suptitle(subtitle, fontsize=16, fontweight='bold')
pl.subplots_adjust(hspace=0.5, left=0.1, right=0.8)

pl.show()

---
### Cycle Emittance Violins
---

In [ ]:
dict_intervals_two_beams = filln_CycleDict
t_start_fill, t_end_fill, t_fill_len, t_ref = getCycleDataTimes(bmodes, filln)

Delta_t_b1_inj = (np.array(dict_intervals_two_beams['beam_1']['Injection']['at_end']['time_meas'])-np.array(dict_intervals_two_beams['beam_1']['Injection']['at_start']['time_meas']))
Delta_t_b2_inj = (np.array(dict_intervals_two_beams['beam_2']['Injection']['at_end']['time_meas'])-np.array(dict_intervals_two_beams['beam_2']['Injection']['at_start']['time_meas']))
Delta_t_b1_he = (np.array(dict_intervals_two_beams['beam_1']['he_before_SB']['at_end']['time_meas'])-np.array(dict_intervals_two_beams['beam_1']['he_before_SB']['at_start']['time_meas']))
Delta_t_b2_he = (np.array(dict_intervals_two_beams['beam_2']['he_before_SB']['at_end']['time_meas'])-np.array(dict_intervals_two_beams['beam_2']['he_before_SB']['at_start']['time_meas']))

# Plotting bunches that stayed at least 5min
mask_b1_inj = Delta_t_b1_inj > 5.0*60.0
mask_b2_inj = Delta_t_b2_inj > 5.0*60.0
mask_b1_he = Delta_t_b1_he > 5.0*60.0
mask_b2_he = Delta_t_b2_he > 5.0*60.0

b1h_INJ = np.array(dict_intervals_two_beams['beam_1']['Injection']['at_start']['emith'])[mask_b1_inj]
b1h_FB  = np.array(dict_intervals_two_beams['beam_1']['Injection']['at_end']['emith'])[mask_b1_inj]
b1h_FT  = np.array(dict_intervals_two_beams['beam_1']['he_before_SB']['at_start']['emith'])[mask_b1_he]
b1h_SB  = np.array(dict_intervals_two_beams['beam_1']['he_before_SB']['at_end']['emith'])[mask_b1_he]

b1v_INJ = np.array(dict_intervals_two_beams['beam_1']['Injection']['at_start']['emitv'])[mask_b1_inj]
b1v_FB  = np.array(dict_intervals_two_beams['beam_1']['Injection']['at_end']['emitv'])[mask_b1_inj]
b1v_FT  = np.array(dict_intervals_two_beams['beam_1']['he_before_SB']['at_start']['emitv'])[mask_b1_he]
b1v_SB  = np.array(dict_intervals_two_beams['beam_1']['he_before_SB']['at_end']['emitv'])[mask_b1_he]


b2h_INJ = np.array(dict_intervals_two_beams['beam_2']['Injection']['at_start']['emith'])[mask_b2_inj]
b2h_FB  = np.array(dict_intervals_two_beams['beam_2']['Injection']['at_end']['emith'])[mask_b2_inj]
b2h_FT  = np.array(dict_intervals_two_beams['beam_2']['he_before_SB']['at_start']['emith'])[mask_b2_he]
b2h_SB  = np.array(dict_intervals_two_beams['beam_2']['he_before_SB']['at_end']['emith'])[mask_b2_he]


b2v_INJ = np.array(dict_intervals_two_beams['beam_2']['Injection']['at_start']['emitv'])[mask_b2_inj]
b2v_FB  = np.array(dict_intervals_two_beams['beam_2']['Injection']['at_end']['emitv'])[mask_b2_inj]
b2v_FT  = np.array(dict_intervals_two_beams['beam_2']['he_before_SB']['at_start']['emitv'])[mask_b2_he]
b2v_SB  = np.array(dict_intervals_two_beams['beam_2']['he_before_SB']['at_end']['emitv'])[mask_b2_he]

b1h_emit = [b1h_INJ, b1h_FB, b1h_FT, b1h_SB]
b1v_emit = [b1h_INJ, b1v_FB, b1v_FT, b1v_SB]

b2h_emit = [b2h_INJ, b2h_FB, b2h_FT, b2h_SB]
b2v_emit = [b2v_INJ, b2v_FB, b2v_FT, b2v_SB]

emit  = []
cycle = []
beam  = []
plane = []
slot  = []

# start with b1h
emit.append(b1h_INJ)
cycle.append(['Injection']*len(b1h_INJ))
beam.append(['B1']*len(b1h_INJ))
plane.append(['Horizontal']*len(b1h_INJ))
slot.append(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_1']['Injection']['filled_slots'])).tolist())

emit.append(b1h_FB)
cycle.append(['Start Ramp']*len(b1h_FB))
beam.append(['B1']*len(b1h_FB))
plane.append(['Horizontal']*len(b1h_FB))
slot.append(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_1']['Injection']['filled_slots'])).tolist())

emit.append(b1h_FT)
cycle.append(['End Ramp']*len(b1h_FT))
beam.append(['B1']*len(b1h_FT))
plane.append(['Horizontal']*len(b1h_FT))
slot.append(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_1']['he_before_SB']['filled_slots'])).tolist())

emit.append(b1h_SB)
cycle.append(['Stable Beams']*len(b1h_SB))
beam.append(['B1']*len(b1h_SB))
plane.append(['Horizontal']*len(b1h_SB))
slot.append(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_1']['he_before_SB']['filled_slots'])).tolist())

# b1v
emit.append(b1v_INJ)
cycle.append(['Injection']*len(b1v_INJ))
beam.append(['B1']*len(b1v_INJ))
plane.append(['Vertical']*len(b1v_INJ))
slot.append(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_1']['Injection']['filled_slots'])).tolist())

emit.append(b1v_FB)
cycle.append(['Start Ramp']*len(b1v_FB))
beam.append(['B1']*len(b1v_FB))
plane.append(['Vertical']*len(b1v_FB))
slot.append(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_1']['Injection']['filled_slots'])).tolist())

emit.append(b1v_FT)
cycle.append(['End Ramp']*len(b1v_FT))
beam.append(['B1']*len(b1v_FT))
plane.append(['Vertical']*len(b1v_FT))
slot.append(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_1']['he_before_SB']['filled_slots'])).tolist())

emit.append(b1v_SB)
cycle.append(['Stable Beams']*len(b1v_SB))
beam.append(['B1']*len(b1v_SB))
plane.append(['Vertical']*len(b1v_SB))
slot.append(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_1']['he_before_SB']['filled_slots'])).tolist())

# b2h
emit.append(b2h_INJ)
cycle.append(['Injection']*len(b2h_INJ))
beam.append(['B2']*len(b2h_INJ))
plane.append(['Horizontal']*len(b2h_INJ))
slot.append(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_2']['Injection']['filled_slots'])).tolist())


emit.append(b2h_FB)
cycle.append(['Start Ramp']*len(b2h_FB))
beam.append(['B2']*len(b2h_FB))
plane.append(['Horizontal']*len(b2h_FB))
slot.append(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_2']['Injection']['filled_slots'])).tolist())

emit.append(b2h_FT)
cycle.append(['End Ramp']*len(b2h_FT))
beam.append(['B2']*len(b2h_FT))
plane.append(['Horizontal']*len(b2h_FT))
slot.append(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_2']['he_before_SB']['filled_slots'])).tolist())

emit.append(b2h_SB)
cycle.append(['Stable Beams']*len(b2h_SB))
beam.append(['B2']*len(b2h_SB))
plane.append(['Horizontal']*len(b2h_SB))
slot.append(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_2']['he_before_SB']['filled_slots'])).tolist())

# b2v
emit.append(b2v_INJ)
cycle.append(['Injection']*len(b2v_INJ))
beam.append(['B2']*len(b2v_INJ))
plane.append(['Vertical']*len(b2v_INJ))
slot.append(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_2']['Injection']['filled_slots'])).tolist())

emit.append(b2v_FB)
cycle.append(['Start Ramp']*len(b2v_FB))
beam.append(['B2']*len(b2v_FB))
plane.append(['Vertical']*len(b2v_FB))
slot.append(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_2']['Injection']['filled_slots'])).tolist())

emit.append(b2v_FT)
cycle.append(['End Ramp']*len(b2v_FT))
beam.append(['B2']*len(b2v_FT))
plane.append(['Vertical']*len(b2v_FT))
slot.append(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_2']['he_before_SB']['filled_slots'])).tolist())

emit.append(b2v_SB)
cycle.append(['Stable Beams']*len(b2v_SB))
beam.append(['B2']*len(b2v_SB))
plane.append(['Vertical']*len(b2v_SB))
slot.append(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_2']['he_before_SB']['filled_slots'])).tolist())

emit = list(itertools.chain.from_iterable(emit))
cycle = list(itertools.chain.from_iterable(cycle))
beam = list(itertools.chain.from_iterable(beam))
plane = list(itertools.chain.from_iterable(plane))
slot = list(itertools.chain.from_iterable(slot))

df_emit = pd.DataFrame()
df_emit['Emittance'] = pd.Series(emit, dtype='float')
df_emit['Cycle']     = pd.Series(cycle, dtype='category')
df_emit['Plane']     = pd.Series(plane, dtype='category')
df_emit['Beam']      = pd.Series(beam, dtype='category')
df_emit['Slot']      = pd.Series(slot, dtype='int')
df_emit['Fill']      = [int(filln)]*len(df_emit)
df_emit['bunches']   = [int(my_bunches)]*len(df_emit)

df_emit = df_emit.dropna(axis=0, how='any')


mean_values_b1 = []
for ncycle in ['Injection', 'Start Ramp', 'End Ramp', 'Stable Beams']:
    for nplane in ['Horizontal', 'Vertical']:
        mean_values_b1.append(df_emit['Emittance'][(df_emit['Cycle']==ncycle) & (df_emit['Plane']==nplane) & (df_emit['Beam']=='B1')].mean())
mean_values_b2 = []
for ncycle in ['Injection', 'Start Ramp', 'End Ramp', 'Stable Beams']:
    for nplane in ['Horizontal', 'Vertical']:
        mean_values_b2.append(df_emit['Emittance'][(df_emit['Cycle']==ncycle) & (df_emit['Plane']==nplane) & (df_emit['Beam']=='B2')].mean())

mean_labels_b1 = [str(np.round(s, 2))+'$\mu$m' for s in mean_values_b1]
mean_labels_b2 = [str(np.round(s, 2))+'$\mu$m' for s in mean_values_b2] 
pos = range(len(mean_labels_b1))

sns.set_style("whitegrid")


fig_emit_volin = pl.figure('Violins', figsize=(15,8))
ax_b1 = pl.subplot(211)
ax_b1 = sns.violinplot(x="Cycle", y="Emittance", hue="Plane", data=df_emit[df_emit['Beam']=='B1'], palette=['#71aaf2', '#ff8a3d'], split=True, order=['Injection', 'Start Ramp', 'End Ramp', 'Stable Beams'])
ax_b1.get_axes().xaxis.set_ticklabels([])
ax_b1.set_ylim(1,4)
ax_b1.set_ylabel('B1 $\mathbf{\epsilon_{n}}$ [$\mathbf{\mu}$m]', fontsize=16, fontweight='bold')
ax_b1.set_xlabel('')


ax_b2 = pl.subplot(212)#, sharex=ax_b1h, sharey=ax_b1h)
ax_b2 = sns.violinplot(x="Cycle", y="Emittance", hue="Plane", data=df_emit[df_emit['Beam']=='B2'],  palette=['#71aaf2', '#ff8a3d'], split=True, order=['Injection', 'Start Ramp', 'End Ramp', 'Stable Beams'])
ax_b2.get_axes().xaxis.set_ticklabels([])
ax_b2.set_ylim(1,4)
ax_b2.set_ylabel('B2 $\mathbf{\epsilon_{n}}$ [$\mathbf{\mu}$m]', fontsize=16, fontweight='bold')
ax_b2.get_axes().xaxis.set_ticklabels(['INJECTION', 'Start RAMP', 'End RAMP', 'Start STABLE BEAMS'])
ax_b2.set_xlabel('')
for tick in ax_b2.xaxis.get_major_ticks():
    tick.label.set_fontsize(16)
    tick.label.set_fontweight('bold')

for tick in ax_b1.yaxis.get_major_ticks():
    tick.label.set_fontsize(14)
for tick in ax_b2.yaxis.get_major_ticks():
    tick.label.set_fontsize(14)

ax_b1.legend(loc=2, fontsize=14)
ax_b2.legend(loc=2, fontsize=14)

tref_string = datetime.fromtimestamp(t_ref)
subtitle    = 'Fill {} : Started on {}'.format(filln, tref_string)
fig_emit_volin.suptitle(subtitle, fontsize=16, fontweight='bold')

for tick,label in zip(pos,ax_b1.get_xticklabels()):
    ax_b1.text(pos[tick]-0.15, mean_values_b1[2*tick],   mean_labels_b1[2*tick],    horizontalalignment='center', size='small', color='k', weight='semibold')
    ax_b1.text(pos[tick]+0.15, mean_values_b1[2*tick+1], mean_labels_b1[2*tick+1],  horizontalalignment='center', size='small', color='k', weight='semibold')
for tick,label in zip(pos,ax_b2.get_xticklabels()):
    ax_b2.text(pos[tick]-0.15, mean_values_b2[2*tick],   mean_labels_b2[2*tick], 	 horizontalalignment='center', size='small', color='k', weight='semibold')
    ax_b2.text(pos[tick]+0.15, mean_values_b2[2*tick+1], mean_labels_b2[2*tick+1],  horizontalalignment='center', size='small', color='k', weight='semibold')

---
### Cycle Intensities
---

In [ ]:
dict_intervals_two_beams = filln_CycleDict
t_start_fill, t_end_fill, t_fill_len, t_ref = getCycleDataTimes(bmodes, filln)
#################

fig_bbbintens = pl.figure("Intensities", figsize=(14, 7))
ax_b1 = pl.subplot(2,1,1)


# beam 1
ax_b1.plot(getFilledSlotsArray(dict_intervals_two_beams, "beam_1", "Injection",       "at_start",  mask_invalid=True), ma.masked_invalid(np.array(dict_intervals_two_beams['beam_1']['Injection']['at_start']['intensity'])), '.', color='blue',   markersize=8, label='Injected')
ax_b1.plot(getFilledSlotsArray(dict_intervals_two_beams, "beam_1", "Injection",       "at_end",    mask_invalid=True), ma.masked_invalid(np.array(dict_intervals_two_beams['beam_1']['Injection']['at_end']['intensity']  )),   '.', color='orange', markersize=8, label='Start Ramp')
ax_b1.plot(getFilledSlotsArray(dict_intervals_two_beams, "beam_1", "he_before_SB",    "at_start",    mask_invalid=True), ma.masked_invalid(np.array(dict_intervals_two_beams['beam_1']['he_before_SB']['at_start']['intensity'])),  '.', color='green',  markersize=8, label='End Ramp')
ax_b1.plot(getFilledSlotsArray(dict_intervals_two_beams, "beam_1", "he_before_SB",    "at_end",      mask_invalid=True), ma.masked_invalid(np.array(dict_intervals_two_beams['beam_1']['he_before_SB']['at_end']['intensity']  )),    '.', color='red',    markersize=8, label='Start SB')
ax_b1.set_ylabel("B1 Intensity [ppb]", fontsize=14, fontweight='bold')
ax_b1.minorticks_on()
ax_b1.text(0.5, 0.9, "Injected: {:.2e}$\pm${:.2e} | Start Ramp: {:.2e}$\pm${:.2e} | End Ramp: {:.2e}$\pm${:.2e} | Start SB: {:.2e}$\pm${:.2e}".format(
        np.nanmean(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_1']['Injection']['at_start']['intensity']))), 
        np.nanstd(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_1']['Injection']['at_start']['intensity']))),
        np.nanmean(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_1']['Injection']['at_end']['intensity']  ))), 
        np.nanstd(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_1']['Injection']['at_end']['intensity']  ))),
        np.nanmean(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_1']['he_before_SB']['at_start']['intensity']))), 
        np.nanstd(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_1']['he_before_SB']['at_start']['intensity']))),
        np.nanmean(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_1']['he_before_SB']['at_end']['intensity']  ))), 
        np.nanstd(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_1']['he_before_SB']['at_end']['intensity']  )))), 
        horizontalalignment='center', verticalalignment='top', transform=ax_b1.transAxes,
        bbox=dict(facecolor='white', edgecolor='gray', boxstyle='round,pad=0.5', alpha=0.7), fontweight='bold', fontsize=9)
## Shrink current axis by 20%
box = ax_b1.get_position()
ax_b1.set_position([box.x0, box.y0, box.width*0.9, box.height])
# Put a legend to the right of the current axis
ax_b1.legend(loc='center left', bbox_to_anchor=(1, 0.5), fontsize=14, numpoints=1)
ax_b1.grid('on', which='both')
ax_b1.set_ylim(0.5e11, 1.7e11)

# beam 2
ax_b2 = pl.subplot(2,1,2)
ax_b2.plot(getFilledSlotsArray(dict_intervals_two_beams, "beam_2", "Injection",       "at_start",  mask_invalid=True), ma.masked_invalid(np.array(dict_intervals_two_beams['beam_2']['Injection']['at_start']['intensity'])), '.', color='blue',   markersize=8, label='Injected')
ax_b2.plot(getFilledSlotsArray(dict_intervals_two_beams, "beam_2", "Injection",       "at_end",    mask_invalid=True), ma.masked_invalid(np.array(dict_intervals_two_beams['beam_2']['Injection']['at_end']['intensity']  )),   '.', color='orange', markersize=8, label='Start Ramp')
ax_b2.plot(getFilledSlotsArray(dict_intervals_two_beams, "beam_2", "he_before_SB",    "at_start",    mask_invalid=True), ma.masked_invalid(np.array(dict_intervals_two_beams['beam_2']['he_before_SB']['at_start']['intensity'])),  '.', color='green',  markersize=8, label='End Ramp')
ax_b2.plot(getFilledSlotsArray(dict_intervals_two_beams, "beam_2", "he_before_SB",    "at_end",      mask_invalid=True), ma.masked_invalid(np.array(dict_intervals_two_beams['beam_2']['he_before_SB']['at_end']['intensity']  )),    '.', color='red',    markersize=8, label='Start SB')
ax_b2.set_ylabel("B2 Intensity [ppb]", fontsize=14, fontweight='bold')
ax_b2.set_xlabel("Bunch Slots [25ns]", fontsize=14, fontweight='bold')
ax_b2.minorticks_on()
ax_b2.text(0.5, 0.9, "Injected: {:.2e}$\pm${:.2e} | Start Ramp: {:.2e}$\pm${:.2e} | End Ramp: {:.2e}$\pm${:.2e} | Start SB: {:.2e}$\pm${:.2e}".format(
        np.nanmean(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_2']['Injection']['at_start']['intensity']))), 
        np.nanstd(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_2']['Injection']['at_start']['intensity']))),
        np.nanmean(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_2']['Injection']['at_end']['intensity']  ))), 
        np.nanstd(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_2']['Injection']['at_end']['intensity']  ))),
        np.nanmean(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_2']['he_before_SB']['at_start']['intensity']))), 
        np.nanstd(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_2']['he_before_SB']['at_start']['intensity']))),
        np.nanmean(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_2']['he_before_SB']['at_end']['intensity']  ))), 
        np.nanstd(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_2']['he_before_SB']['at_end']['intensity']  )))), 
        horizontalalignment='center', verticalalignment='center', transform=ax_b2.transAxes,
        bbox=dict(facecolor='white', edgecolor='gray', boxstyle='round,pad=0.5', alpha=0.7), fontweight='bold', fontsize=9)

## Shrink current axis by 20%
box = ax_b2.get_position()
ax_b2.set_position([box.x0, box.y0, box.width*0.9, box.height])
# Put a legend to the right of the current axis
ax_b2.legend(loc='center left', bbox_to_anchor=(1, 0.5), fontsize=14, numpoints=1)
ax_b2.grid('on', which='both')
ax_b2.set_ylim(0.5e11, 1.7e11)
# tref string
tref_string = datetime.fromtimestamp(t_ref)
subtitle    = 'Fill {} : Started on {}'.format(filln, tref_string)

fig_bbbintens.suptitle(subtitle, fontsize=16, fontweight='bold')

---
### Cycle Brightness
---

In [ ]:
dict_intervals_two_beams = filln_CycleDict
t_start_fill, t_end_fill, t_fill_len, t_ref = getCycleDataTimes(bmodes, filln)

fig_bbbbright = pl.figure("Brightness", figsize=(14, 7))
fig_bbbbright.set_facecolor('w')
ax_b1 = pl.subplot(2,1,1)

# beam 1
ax_b1.plot(getFilledSlotsArray(dict_intervals_two_beams, "beam_1", "Injection",       "at_start",  mask_invalid=True), ma.masked_invalid(np.array(dict_intervals_two_beams['beam_1']['Injection']['at_start']['brightness'])), '.', color='blue',   markersize=8, label='Injected')
ax_b1.plot(getFilledSlotsArray(dict_intervals_two_beams, "beam_1", "Injection",       "at_end",    mask_invalid=True), ma.masked_invalid(np.array(dict_intervals_two_beams['beam_1']['Injection']['at_end']['brightness'])),   '.', color='orange', markersize=8, label='Start Ramp')
ax_b1.plot(getFilledSlotsArray(dict_intervals_two_beams, "beam_1", "he_before_SB",    "at_start",  mask_invalid=True), ma.masked_invalid(np.array(dict_intervals_two_beams['beam_1']['he_before_SB']['at_start']['brightness'])),  '.', color='green',  markersize=8, label='End Ramp')
ax_b1.plot(getFilledSlotsArray(dict_intervals_two_beams, "beam_1", "he_before_SB",    "at_end",    mask_invalid=True), ma.masked_invalid(np.array(dict_intervals_two_beams['beam_1']['he_before_SB']['at_end']['brightness'])),    '.', color='red',    markersize=8, label='Start SB')
ax_b1.set_ylabel("B1 Brightness [p/$\mathbf{\mu}$m]", fontsize=14, fontweight='bold')
ax_b1.minorticks_on()
ax_b1.text(0.5, 0.9, "Injected: {:.2e}$\pm${:.2e} | Start Ramp: {:.2e}$\pm${:.2e} | End Ramp: {:.2e}$\pm${:.2e} | Start SB: {:.2e}$\pm${:.2e}".format(
        np.nanmean(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_1']['Injection']['at_start']['brightness']))), 
        np.nanstd(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_1']['Injection']['at_start']['brightness']))),
        np.nanmean(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_1']['Injection']['at_end']['brightness']))), 
        np.nanstd(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_1']['Injection']['at_end']['brightness']))),
        np.nanmean(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_1']['he_before_SB']['at_start']['brightness']))), 
        np.nanstd(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_1']['he_before_SB']['at_start']['brightness']))),
        np.nanmean(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_1']['he_before_SB']['at_end']['brightness']))), 
        np.nanstd(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_1']['he_before_SB']['at_end']['brightness'])))), 
        horizontalalignment='center', verticalalignment='top', transform=ax_b1.transAxes,
        bbox=dict(facecolor='white', edgecolor='gray', boxstyle='round,pad=0.5', alpha=0.7), fontweight='bold', fontsize=9)
## Shrink current axis by 20%
box = ax_b1.get_position()
ax_b1.set_position([box.x0, box.y0, box.width*0.9, box.height])
# Put a legend to the right of the current axis
ax_b1.legend(loc='center left', bbox_to_anchor=(1, 0.5), fontsize=14, numpoints=1)
ax_b1.grid('on', which='both')

# beam 2
ax_b2 = pl.subplot(2,1,2)
ax_b2.plot(getFilledSlotsArray(dict_intervals_two_beams, "beam_2", "Injection",       "at_start",  mask_invalid=True), ma.masked_invalid(np.array(dict_intervals_two_beams['beam_2']['Injection']['at_start']['brightness'])), '.', color='blue',   markersize=8, label='Injected')
ax_b2.plot(getFilledSlotsArray(dict_intervals_two_beams, "beam_2", "Injection",       "at_end",    mask_invalid=True), ma.masked_invalid(np.array(dict_intervals_two_beams['beam_2']['Injection']['at_end']['brightness'])),   '.', color='orange', markersize=8, label='Start Ramp')
ax_b2.plot(getFilledSlotsArray(dict_intervals_two_beams, "beam_2", "he_before_SB",    "at_start",  mask_invalid=True), ma.masked_invalid(np.array(dict_intervals_two_beams['beam_2']['he_before_SB']['at_start']['brightness'])),  '.', color='green',  markersize=8, label='End Ramp')
ax_b2.plot(getFilledSlotsArray(dict_intervals_two_beams, "beam_2", "he_before_SB",    "at_end",    mask_invalid=True), ma.masked_invalid(np.array(dict_intervals_two_beams['beam_2']['he_before_SB']['at_end']['brightness'])),    '.', color='red',    markersize=8, label='Start SB')
ax_b2.set_ylabel("B2 Brightness [p/$\mathbf{\mu}$m]", fontsize=14, fontweight='bold')
ax_b2.set_xlabel("Bunch Slots [25ns]", fontsize=14, fontweight='bold')
ax_b2.minorticks_on()
ax_b2.text(0.5, 0.9, "Injected: {:.2e}$\pm${:.2e} | Start Ramp: {:.2e}$\pm${:.2e} | End Ramp: {:.2e}$\pm${:.2e} | Start SB: {:.2e}$\pm${:.2e}".format(
        np.nanmean(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_2']['Injection']['at_start']['brightness']))), 
        np.nanstd(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_2']['Injection']['at_start']['brightness']))),
        np.nanmean(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_2']['Injection']['at_end']['brightness']))), 
        np.nanstd(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_2']['Injection']['at_end']['brightness']))),
        np.nanmean(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_2']['he_before_SB']['at_start']['brightness']))), 
        np.nanstd(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_2']['he_before_SB']['at_start']['brightness']))),
        np.nanmean(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_2']['he_before_SB']['at_end']['brightness']))), 
        np.nanstd(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_2']['he_before_SB']['at_end']['brightness'])))), 
        horizontalalignment='center', verticalalignment='center', transform=ax_b2.transAxes,
        bbox=dict(facecolor='white', edgecolor='gray', boxstyle='round,pad=0.5', alpha=0.7), fontweight='bold', fontsize=9)
## Shrink current axis by 20%
box = ax_b2.get_position()
ax_b2.set_position([box.x0, box.y0, box.width*0.9, box.height])
# Put a legend to the right of the current axis
ax_b2.legend(loc='center left', bbox_to_anchor=(1, 0.5), fontsize=14, numpoints=1)
ax_b2.grid('on', which='both')
# tref string
tref_string = datetime.fromtimestamp(t_ref)
subtitle    = 'Fill {} : Started on {}'.format(filln, tref_string)
fig_bbbbright.suptitle(subtitle, fontsize=16, fontweight='bold')

---
### Cycle Bunch Length
---

In [ ]:
dict_intervals_two_beams = filln_CycleDict
t_start_fill, t_end_fill, t_fill_len, t_ref = getCycleDataTimes(bmodes, filln)

fig_bbbblength = pl.figure("blength", figsize=(14, 7))
fig_bbbblength.set_facecolor('w')
ax_b1 = pl.subplot(2,1,1)
# beam 1
ax_b1.plot(getFilledSlotsArray(dict_intervals_two_beams, "beam_1", "Injection",       "at_start",  mask_invalid=True), ma.masked_invalid(np.array(dict_intervals_two_beams['beam_1']['Injection']['at_start']['blength']))/1.0e-9, '.', color='blue',   markersize=8, label='Injected')
ax_b1.plot(getFilledSlotsArray(dict_intervals_two_beams, "beam_1", "Injection",       "at_end",    mask_invalid=True), ma.masked_invalid(np.array(dict_intervals_two_beams['beam_1']['Injection']['at_end']['blength']))/1.0e-9,   '.', color='orange', markersize=8, label='Start Ramp')
ax_b1.plot(getFilledSlotsArray(dict_intervals_two_beams, "beam_1", "he_before_SB",    "at_start",  mask_invalid=True), ma.masked_invalid(np.array(dict_intervals_two_beams['beam_1']['he_before_SB']['at_start']['blength']))/1.0e-9,  '.', color='green',  markersize=8, label='End Ramp')
ax_b1.plot(getFilledSlotsArray(dict_intervals_two_beams, "beam_1", "he_before_SB",    "at_end",    mask_invalid=True), ma.masked_invalid(np.array(dict_intervals_two_beams['beam_1']['he_before_SB']['at_end']['blength']))/1.0e-9,    '.', color='red',    markersize=8, label='Start SB')
ax_b1.set_ylabel("B1 Bunch Length [ns]", fontsize=12, fontweight='bold')
ax_b1.minorticks_on()
ax_b1.text(0.5, 0.1, "Injected: {:.2e}$\pm${:.2e} | Start Ramp: {:.2e}$\pm${:.2e} | End Ramp: {:.2e}$\pm${:.2e} | Start SB: {:.2e}$\pm${:.2e}".format(
    np.nanmean(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_1']['Injection']['at_start']['blength']))), 
    np.nanstd(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_1']['Injection']['at_start']['blength']))),
    np.nanmean(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_1']['Injection']['at_end']['blength']))), 
    np.nanstd(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_1']['Injection']['at_end']['blength']))),
    np.nanmean(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_1']['he_before_SB']['at_start']['blength']))), 
    np.nanstd(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_1']['he_before_SB']['at_start']['blength']))),
    np.nanmean(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_1']['he_before_SB']['at_end']['blength']))), 
    np.nanstd(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_1']['he_before_SB']['at_end']['blength'])))), 
    horizontalalignment='center', verticalalignment='top', transform=ax_b1.transAxes,
    bbox=dict(facecolor='white', edgecolor='gray', boxstyle='round,pad=0.5', alpha=0.7), fontweight='bold', fontsize=9)

## Shrink current axis by 20%
box = ax_b1.get_position()
ax_b1.set_position([box.x0, box.y0, box.width*0.9, box.height])
# Put a legend to the right of the current axis
ax_b1.legend(loc='center left', bbox_to_anchor=(1, 0.5), fontsize=14, numpoints=1)
ax_b1.grid('on', which='both')
ax_b1.set_ylim(0.5, 1.5)
# beam 2
ax_b2 = pl.subplot(2,1,2)
ax_b2.plot(getFilledSlotsArray(dict_intervals_two_beams, "beam_2", "Injection",       "at_start",  mask_invalid=True), ma.masked_invalid(np.array(dict_intervals_two_beams['beam_2']['Injection']['at_start']['blength']))/1.0e-9, '.', color='blue',   markersize=8, label='Injected')
ax_b2.plot(getFilledSlotsArray(dict_intervals_two_beams, "beam_2", "Injection",       "at_end",    mask_invalid=True), ma.masked_invalid(np.array(dict_intervals_two_beams['beam_2']['Injection']['at_end']['blength']))/1.0e-9,   '.', color='orange', markersize=8, label='Start Ramp')
ax_b2.plot(getFilledSlotsArray(dict_intervals_two_beams, "beam_2", "he_before_SB",    "at_start",  mask_invalid=True), ma.masked_invalid(np.array(dict_intervals_two_beams['beam_2']['he_before_SB']['at_start']['blength']))/1.0e-9,  '.', color='green',  markersize=8, label='End Ramp')
ax_b2.plot(getFilledSlotsArray(dict_intervals_two_beams, "beam_2", "he_before_SB",    "at_end",    mask_invalid=True), ma.masked_invalid(np.array(dict_intervals_two_beams['beam_2']['he_before_SB']['at_end']['blength']))/1.0e-9,    '.', color='red',    markersize=8, label='Start SB')
ax_b2.set_ylabel("B2 Bunch Length [ns]", fontsize=12, fontweight='bold')
ax_b2.set_xlabel("Bunch Slots [25ns]", fontsize=14, fontweight='bold')
ax_b2.minorticks_on()
ax_b2.text(0.5, 0.1, "Injected: {:.2e}$\pm${:.2e} | Start Ramp: {:.2e}$\pm${:.2e} | End Ramp: {:.2e}$\pm${:.2e} | Start SB: {:.2e}$\pm${:.2e}".format(
    np.nanmean(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_2']['Injection']['at_start']['blength']))), 
    np.nanstd(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_2']['Injection']['at_start']['blength']))),
    np.nanmean(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_2']['Injection']['at_end']['blength']))), 
    np.nanstd(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_2']['Injection']['at_end']['blength']))),
    np.nanmean(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_2']['he_before_SB']['at_start']['blength']))), 
    np.nanstd(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_2']['he_before_SB']['at_start']['blength']))),
    np.nanmean(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_2']['he_before_SB']['at_end']['blength']))), 
    np.nanstd(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_2']['he_before_SB']['at_end']['blength'])))), 
    horizontalalignment='center', verticalalignment='center', transform=ax_b2.transAxes,
    bbox=dict(facecolor='white', edgecolor='gray', boxstyle='round,pad=0.5', alpha=0.7), fontweight='bold', fontsize=9)
## Shrink current axis by 20%
box = ax_b2.get_position()
ax_b2.set_position([box.x0, box.y0, box.width*0.9, box.height])
# Put a legend to the right of the current axis
ax_b2.legend(loc='center left', bbox_to_anchor=(1, 0.5), fontsize=14, numpoints=1)
ax_b2.grid('on', which='both')
ax_b2.set_ylim(0.5, 1.5)

# tref string
tref_string = datetime.fromtimestamp(t_ref)
subtitle    = 'Fill {} : Started on {}'.format(filln, tref_string)
fig_bbbblength.suptitle(subtitle, fontsize=16, fontweight='bold')

---
### Cycle Bunch Time
---

In [ ]:
dict_intervals_two_beams = filln_CycleDict

t_start_fill, t_end_fill, t_fill_len, t_ref = getCycleDataTimes(bmodes, filln)

fig_bbbtime = pl.figure("time", figsize=(14, 7))
fig_bbbtime.set_facecolor('w')
# injection
time_b1_inj_atStart = np.array(dict_intervals_two_beams['beam_1']['Injection']['at_start']['time_meas'])
time_b2_inj_atStart = np.array(dict_intervals_two_beams['beam_2']['Injection']['at_start']['time_meas'])
time_b1_inj_atEnd   = np.array(dict_intervals_two_beams['beam_1']['Injection']['at_end']['time_meas'])
time_b2_inj_atEnd   = np.array(dict_intervals_two_beams['beam_2']['Injection']['at_end']['time_meas'])

# flattop
time_b1_ft_atStart = np.array(dict_intervals_two_beams['beam_1']['he_before_SB']['at_start']['time_meas'])
time_b2_ft_atStart = np.array(dict_intervals_two_beams['beam_2']['he_before_SB']['at_start']['time_meas'])
time_b1_ft_atEnd   = np.array(dict_intervals_two_beams['beam_1']['he_before_SB']['at_end']['time_meas'])
time_b2_ft_atEnd   = np.array(dict_intervals_two_beams['beam_2']['he_before_SB']['at_end']['time_meas'])

time_b1_inj = (time_b1_inj_atEnd - time_b1_inj_atStart)/60.
time_b1_ft  = (time_b1_ft_atEnd  - time_b1_ft_atStart)/60.
time_b2_inj = (time_b2_inj_atEnd - time_b2_inj_atStart)/60.
time_b2_ft  = (time_b2_ft_atEnd  - time_b2_ft_atStart)/60.

mean_time_b1_inj = np.nanmean(time_b1_inj)
mean_time_b1_ft  = np.nanmean(time_b1_ft)
mean_time_b2_inj = np.nanmean(time_b2_inj)
mean_time_b2_ft  = np.nanmean(time_b2_ft)

std_time_b1_inj = (np.nanstd(time_b1_inj_atStart) + np.nanstd(time_b1_inj_atEnd))/60.
std_time_b1_ft  = (np.nanstd(time_b1_ft_atStart) + np.nanstd(time_b1_ft_atEnd))/60.

std_time_b2_inj = (np.nanstd(time_b2_inj_atStart) + np.nanstd(time_b2_inj_atEnd))/60.
std_time_b2_ft  = (np.nanstd(time_b2_ft_atStart) + np.nanstd(time_b2_ft_atEnd))/60.

ax_b1 = pl.subplot(2,1,1)

# beam 1
ax_b1.plot(getFilledSlotsArray(dict_intervals_two_beams, "beam_1", "Injection",       "at_start",  mask_invalid=True), time_b1_inj, '.', color='blue',   markersize=8, label='Injection')
ax_b1.plot(getFilledSlotsArray(dict_intervals_two_beams, "beam_1", "he_before_SB",    "at_end",      mask_invalid=True), time_b1_ft,    '.', color='red',    markersize=8, label='Flat Top')
ax_b1.set_ylabel("B1 Time [min]", fontsize=12, fontweight='bold')
ax_b1.minorticks_on()
ax_b1.text(0.5, 0.9, "Injection: {:.2f}$\pm${:.2f} | Flat Top : {:.2f}$\pm${:.2f}".format(mean_time_b1_inj, std_time_b1_inj,mean_time_b1_ft,std_time_b1_ft ), 
           horizontalalignment='center', verticalalignment='top', transform=ax_b1.transAxes, bbox=dict(facecolor='white', edgecolor='gray', boxstyle='round,pad=0.5', alpha=0.7), fontweight='bold', fontsize=12)

## Shrink current axis by 20%
box = ax_b1.get_position()
ax_b1.set_position([box.x0, box.y0, box.width*0.9, box.height])
# Put a legend to the right of the current axis
ax_b1.legend(loc='center left', bbox_to_anchor=(1, 0.5), fontsize=14, numpoints=1)
ax_b1.grid('on', which='both')


# beam 2
ax_b2 = pl.subplot(2,1,2)
ax_b2.plot(getFilledSlotsArray(dict_intervals_two_beams, "beam_2", "Injection",  "at_start",  mask_invalid=True), (time_b2_inj_atEnd-time_b2_inj_atStart)/60., '.', color='blue',   markersize=8, label='Injection')
ax_b2.plot(getFilledSlotsArray(dict_intervals_two_beams, "beam_2", "he_before_SB",    "at_end",      mask_invalid=True), (time_b2_ft_atEnd-time_b2_ft_atStart)/60.,    '.', color='red',    markersize=8, label='Flat Top')
ax_b2.set_ylabel("B2 Time [min]", fontsize=12, fontweight='bold')
ax_b2.set_xlabel("Bunch Slots [25ns]", fontsize=14, fontweight='bold')
ax_b2.minorticks_on()
ax_b2.text(0.5, 0.9, "Injection: {:.2f}$\pm${:.2f} | Flat Top : {:.2f}$\pm${:.2f}".format(mean_time_b2_inj, std_time_b2_inj,mean_time_b2_ft,std_time_b2_ft ),
        horizontalalignment='center', verticalalignment='top', transform=ax_b2.transAxes,
        bbox=dict(facecolor='white', edgecolor='gray', boxstyle='round,pad=0.5', alpha=0.7), fontweight='bold', fontsize=12)

## Shrink current axis by 20%
box = ax_b2.get_position()
ax_b2.set_position([box.x0, box.y0, box.width*0.9, box.height])
# Put a legend to the right of the current axis
ax_b2.legend(loc='center left', bbox_to_anchor=(1, 0.5), fontsize=14, numpoints=1)
ax_b2.grid('on', which='both')

# tref string
tref_string = datetime.fromtimestamp(t_ref)
subtitle    = 'Fill {} : Started on {}'.format(filln, tref_string)

fig_bbbtime.suptitle(subtitle, fontsize=16, fontweight='bold')


---
### Cycle Histos
---

In [ ]:
dict_intervals_two_beams = filln_CycleDict

t_start_fill, t_end_fill, t_fill_len, t_ref = getCycleDataTimes(bmodes, filln)

fig_hist = pl.figure("Histograms", figsize=(18, 9))
fig_hist.clf()
fig_hist.set_facecolor('w')

ax_emit_b1_h = pl.subplot(4,2,1)
ax_emit_b1_h.hist(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_1']['Injection']['at_start']['emith'])),    range=(0, 5),   bins=50, color='blue',   histtype='step', lw=2,  label='Injected')
ax_emit_b1_h.hist(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_1']['Injection']['at_end']['emith'])),      range=(0, 5),   bins=50, color='orange', histtype='step', lw=2,  label='Start Ramp')
ax_emit_b1_h.hist(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_1']['he_before_SB']['at_start']['emith'])),     range=(0, 5),   bins=50, color='green',  histtype='step', lw=2,  label='End Ramp')
ax_emit_b1_h.hist(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_1']['he_before_SB']['at_end']['emith'])),       range=(0, 5),   bins=50, color='red',    histtype='step', lw=2,  label='Start SB')
ax_emit_b1_h.set_ylabel('Entries', fontweight='bold', fontsize=8)
ax_emit_b1_h.set_xlabel('B1 $\mathbf{\epsilon_{H}}$ [$\mathbf{\mu}$m]', fontweight='bold', fontsize=8)
## Shrink current axis by 20%
box = ax_emit_b1_h.get_position()
ax_emit_b1_h.set_position([box.x0, box.y0, box.width*0.9, box.height])
# Put a legend to the right of the current axis
ax_emit_b1_h.legend(loc='center left', bbox_to_anchor=(1, 0.5), fontsize=12)
ax_emit_b1_h.grid('on', which='both')
ax_emit_b1_h.text(0.5, 1.1, "Injected: {:.2f}$\pm${:.2f} | Start Ramp: {:.2f}$\pm${:.2f} | End Ramp: {:.2f}$\pm${:.2f} | Start SB: {:.2f}$\pm${:.2f}".format(
            np.nanmean(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_1']['Injection']['at_start']['emith']))), 
            np.nanstd(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_1']['Injection']['at_start']['emith']))),
            np.nanmean(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_1']['Injection']['at_end']['emith']))), 
            np.nanstd(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_1']['Injection']['at_end']['emith']))),
            np.nanmean(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_1']['he_before_SB']['at_start']['emith']))), 
            np.nanstd(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_1']['he_before_SB']['at_start']['emith']))),
            np.nanmean(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_1']['he_before_SB']['at_end']['emith']))), 
            np.nanstd(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_1']['he_before_SB']['at_end']['emith'])))), 
            horizontalalignment='center', verticalalignment='center', transform=ax_emit_b1_h.transAxes,
            bbox=dict(facecolor='white', edgecolor='gray', boxstyle='round,pad=0.5', alpha=0.7), fontweight='bold', fontsize=7)


ax_emit_b1_v = pl.subplot(4,2,3, sharex=ax_emit_b1_h , sharey=ax_emit_b1_h)
ax_emit_b1_v.hist(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_1']['Injection']['at_start']['emitv'])),    range=(0, 5),   bins=50, color='blue',    histtype='step', lw=2, label='Injected')
ax_emit_b1_v.hist(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_1']['Injection']['at_end']['emitv'])),      range=(0, 5),   bins=50, color='orange',  histtype='step', lw=2, label='Start Ramp')
ax_emit_b1_v.hist(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_1']['he_before_SB']['at_start']['emitv'])),     range=(0, 5),   bins=50, color='green',   histtype='step', lw=2, label='End Ramp')
ax_emit_b1_v.hist(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_1']['he_before_SB']['at_end']['emitv'])),       range=(0, 5),   bins=50, color='red',     histtype='step', lw=2, label='Start SB')
ax_emit_b1_v.set_ylabel('Entries', fontweight='bold', fontsize=8)
ax_emit_b1_v.set_xlabel('B1 $\mathbf{\epsilon_{V}}$ [$\mathbf{\mu}$m]', fontweight='bold', fontsize=8)
ax_emit_b1_v.text(0.5, 1.1, "Injected: {:.2f}$\pm${:.2f} | Start Ramp: {:.2f}$\pm${:.2f} | End Ramp: {:.2f}$\pm${:.2f} | Start SB: {:.2f}$\pm${:.2f}".format(
    np.nanmean(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_1']['Injection']['at_start']['emitv']))),
    np.nanstd(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_1']['Injection']['at_start']['emitv']))),
    np.nanmean(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_1']['Injection']['at_end']['emitv']))),  
    np.nanstd(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_1']['Injection']['at_end']['emitv']))),
    np.nanmean(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_1']['he_before_SB']['at_start']['emitv']))), 
    np.nanstd(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_1']['he_before_SB']['at_start']['emitv']))),
    np.nanmean(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_1']['he_before_SB']['at_end']['emitv']))),   
    np.nanstd(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_1']['he_before_SB']['at_end']['emitv'])))), 
    horizontalalignment='center', verticalalignment='center', transform=ax_emit_b1_v.transAxes,
    bbox=dict(facecolor='white', edgecolor='gray', boxstyle='round,pad=0.5', alpha=0.7), fontweight='bold', fontsize=7)
box = ax_emit_b1_v.get_position()
ax_emit_b1_v.set_position([box.x0, box.y0, box.width*0.9, box.height])
ax_emit_b1_v.legend(loc='center left', bbox_to_anchor=(1, 0.5), fontsize=12)
ax_emit_b1_v.grid('on', which='both')

ax_emit_b2_h = pl.subplot(4,2,5,  sharex=ax_emit_b1_h , sharey=ax_emit_b1_h)
ax_emit_b2_h.hist(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_2']['Injection']['at_start']['emith'])),    range=(0, 5),   bins=50, color='blue',   histtype='step', lw=2, label='Injected')
ax_emit_b2_h.hist(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_2']['Injection']['at_end']['emith'])),      range=(0, 5),   bins=50, color='orange', histtype='step', lw=2, label='Start Ramp')
ax_emit_b2_h.hist(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_2']['he_before_SB']['at_start']['emith'])),     range=(0, 5),   bins=50, color='green',  histtype='step', lw=2, label='End Ramp')
ax_emit_b2_h.hist(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_2']['he_before_SB']['at_end']['emith'])),       range=(0, 5),   bins=50, color='red',    histtype='step', lw=2, label='Start SB')
ax_emit_b2_h.set_ylabel('Entries', fontweight='bold', fontsize=8)
ax_emit_b2_h.set_xlabel('B2 $\mathbf{\epsilon_{H}}$ [$\mathbf{\mu}$m]', fontweight='bold', fontsize=8)
ax_emit_b2_h.text(0.5, 1.1, "Injected: {:.2f}$\pm${:.2f} | Start Ramp: {:.2f}$\pm${:.2f} | End Ramp: {:.2f}$\pm${:.2f} | Start SB: {:.2f}$\pm${:.2f}".format(
    np.nanmean(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_2']['Injection']['at_start']['emith']))), 
    np.nanstd(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_2']['Injection']['at_start']['emith']))),
    np.nanmean(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_2']['Injection']['at_end']['emith']))),  
    np.nanstd(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_2']['Injection']['at_end']['emith']))),
    np.nanmean(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_2']['he_before_SB']['at_start']['emith']))), 
    np.nanstd(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_2']['he_before_SB']['at_start']['emith']))),
    np.nanmean(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_2']['he_before_SB']['at_end']['emith']))),   
    np.nanstd(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_2']['he_before_SB']['at_end']['emith'])))), 
    horizontalalignment='center', verticalalignment='center', transform=ax_emit_b2_h.transAxes,
    bbox=dict(facecolor='white', edgecolor='gray', boxstyle='round,pad=0.5', alpha=0.7), fontweight='bold', fontsize=7)
box = ax_emit_b2_h.get_position()
ax_emit_b2_h.set_position([box.x0, box.y0, box.width*0.9, box.height])
ax_emit_b2_h.legend(loc='center left', bbox_to_anchor=(1, 0.5), fontsize=12)
ax_emit_b2_h.grid('on', which='both')

ax_emit_b2_v = pl.subplot(4,2,7, sharex=ax_emit_b1_h,       sharey=ax_emit_b1_h)
ax_emit_b2_v.hist(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_2']['Injection']['at_start']['emitv'])),    range=(0, 5),   bins=50, color='blue',    histtype='step', lw=2, label='Injected')
ax_emit_b2_v.hist(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_2']['Injection']['at_end']['emitv'])),      range=(0, 5),   bins=50, color='orange',  histtype='step', lw=2, label='Start Ramp')
ax_emit_b2_v.hist(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_2']['he_before_SB']['at_start']['emitv'])),     range=(0, 5),   bins=50, color='green',   histtype='step', lw=2, label='End Ramp')
ax_emit_b2_v.hist(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_2']['he_before_SB']['at_end']['emitv'])),       range=(0, 5),   bins=50, color='red',     histtype='step', lw=2, label='Start SB')
ax_emit_b2_v.set_ylabel('Entries', fontweight='bold', fontsize=8)
ax_emit_b2_v.set_xlabel('B2 $\mathbf{\epsilon_{V}}$ [$\mathbf{\mu}$m]', fontweight='bold', fontsize=8)
ax_emit_b2_v.text(0.5, 1.1, "Injected: {:.2f}$\pm${:.2f} | Start Ramp: {:.2f}$\pm${:.2f} | End Ramp: {:.2f}$\pm${:.2f} | Start SB: {:.2f}$\pm${:.2f}".format(
    np.nanmean(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_2']['Injection']['at_start']['emitv']))), 
    np.nanstd(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_2']['Injection']['at_start']['emitv']))),
    np.nanmean(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_2']['Injection']['at_end']['emitv']))),  
    np.nanstd(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_2']['Injection']['at_end']['emitv']))),
    np.nanmean(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_2']['he_before_SB']['at_start']['emitv']))), 
    np.nanstd(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_2']['he_before_SB']['at_start']['emitv']))),
    np.nanmean(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_2']['he_before_SB']['at_end']['emitv']))),   
    np.nanstd(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_2']['he_before_SB']['at_end']['emitv'])))), 
    horizontalalignment='center', verticalalignment='center', transform=ax_emit_b2_v.transAxes,
    bbox=dict(facecolor='white', edgecolor='gray', boxstyle='round,pad=0.5', alpha=0.7), fontweight='bold', fontsize=7)

box = ax_emit_b2_v.get_position()
ax_emit_b2_v.set_position([box.x0, box.y0, box.width*0.9, box.height])

ax_emit_b2_v.legend(loc='center left', bbox_to_anchor=(1, 0.5), fontsize=12)
ax_emit_b2_v.grid('on', which='both')

##### ==== Intensities
ax_intens_b1 = pl.subplot(4,2,2)
ax_intens_b1.hist(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_1']['Injection']['at_start']['intensity'])),    range=(0.0e11, 1.5e11),   bins=50, color='blue',   histtype='step', lw=2, label='Injected')
ax_intens_b1.hist(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_1']['Injection']['at_end']['intensity'])),      range=(0.0e11, 1.5e11),   bins=50, color='orange', histtype='step', lw=2, label='Start Ramp')
ax_intens_b1.hist(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_1']['he_before_SB']['at_start']['intensity'])),     range=(0.0e11, 1.5e11),   bins=50, color='green',  histtype='step', lw=2, label='End Ramp')
ax_intens_b1.hist(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_1']['he_before_SB']['at_end']['intensity'])),       range=(0.0e11, 1.5e11),   bins=50, color='red',    histtype='step', lw=2, label='Start SB')
ax_intens_b1.set_ylabel('Entries', fontweight='bold', fontsize=8)
ax_intens_b1.set_xlabel('B1 Intensity [ppb]', fontweight='bold', fontsize=8)
ax_intens_b1.text(0.6, 1.1, "Injected: {:.2e}$\pm${:.2e} | Start Ramp: {:.2e}$\pm${:.2e} | End Ramp: {:.2e}$\pm${:.2e} | Start SB: {:.2e}$\pm${:.2e}".format(
    np.nanmean(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_1']['Injection']['at_start']['intensity']))), 
    np.nanstd(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_1']['Injection']['at_start']['intensity']))),
    np.nanmean(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_1']['Injection']['at_end']['intensity']))),  
    np.nanstd(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_1']['Injection']['at_end']['intensity']))),
    np.nanmean(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_1']['he_before_SB']['at_start']['intensity']))), 
    np.nanstd(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_1']['he_before_SB']['at_start']['intensity']))),
    np.nanmean(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_1']['he_before_SB']['at_end']['intensity']))),   
    np.nanstd(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_1']['he_before_SB']['at_end']['intensity'])))), 
    horizontalalignment='center', verticalalignment='center', transform=ax_intens_b1.transAxes,
    bbox=dict(facecolor='white', edgecolor='gray', boxstyle='round,pad=0.5', alpha=0.7), fontweight='bold', fontsize=7)
box = ax_intens_b1.get_position()
ax_intens_b1.set_position([box.x0, box.y0, box.width*0.9, box.height])
ax_intens_b1.legend(loc='center left', bbox_to_anchor=(1, 0.5), fontsize=12)
ax_intens_b1.grid('on', which='both')


ax_intens_b2 = pl.subplot(4,2,4)
ax_intens_b2.hist(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_2']['Injection']['at_start']['intensity'])),    range=(0.0e11, 1.5e11),   bins=50, color='blue',   histtype='step', lw=2, label='Injected')
ax_intens_b2.hist(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_2']['Injection']['at_end']['intensity'])),      range=(0.0e11, 1.5e11),   bins=50, color='orange', histtype='step', lw=2, label='Start Ramp')
ax_intens_b2.hist(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_2']['he_before_SB']['at_start']['intensity'])),     range=(0.0e11, 1.5e11),   bins=50, color='green',  histtype='step', lw=2, label='End Ramp')
ax_intens_b2.hist(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_2']['he_before_SB']['at_end']['intensity'])),       range=(0.0e11, 1.5e11),   bins=50, color='red',    histtype='step', lw=2, label='Start SB')
ax_intens_b2.set_ylabel('Entries', fontweight='bold', fontsize=8)
ax_intens_b2.set_xlabel('B2 Intensity [ppb]', fontweight='bold', fontsize=8)
ax_intens_b2.text(0.6, 1.1, "Injected: {:.2e}$\pm${:.2e} | Start Ramp: {:.2e}$\pm${:.2e} | End Ramp: {:.2e}$\pm${:.2e} | Start SB: {:.2e}$\pm${:.2e}".format(
        np.nanmean(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_2']['Injection']['at_start']['intensity']))), 
        np.nanstd(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_2']['Injection']['at_start']['intensity']))),
        np.nanmean(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_2']['Injection']['at_end']['intensity']))),  
        np.nanstd(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_2']['Injection']['at_end']['intensity']))),
        np.nanmean(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_2']['he_before_SB']['at_start']['intensity']))), 
        np.nanstd(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_2']['he_before_SB']['at_start']['intensity']))),
        np.nanmean(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_2']['he_before_SB']['at_end']['intensity']))),   
        np.nanstd(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_2']['he_before_SB']['at_end']['intensity'])))), 
        horizontalalignment='center', verticalalignment='center', transform=ax_intens_b2.transAxes,
        bbox=dict(facecolor='white', edgecolor='gray', boxstyle='round,pad=0.5', alpha=0.7), fontweight='bold', fontsize=7)

box = ax_intens_b2.get_position()
ax_intens_b2.set_position([box.x0, box.y0, box.width*0.9, box.height])

ax_intens_b2.legend(loc='center left', bbox_to_anchor=(1, 0.5), fontsize=12)
ax_intens_b2.grid('on', which='both')

##### ==== Brightness
ax_bright_b1 = pl.subplot(4,2,6)
ax_bright_b1.hist(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_1']['Injection']['at_start']['brightness'])),    range=(0.0e11, 1.5e11),   bins=50, color='blue',   histtype='step', lw=2, label='Injected')
ax_bright_b1.hist(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_1']['Injection']['at_end']['brightness'])),      range=(0.0e11, 1.5e11),   bins=50, color='orange', histtype='step', lw=2, label='Start Ramp')
ax_bright_b1.hist(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_1']['he_before_SB']['at_start']['brightness'])),     range=(0.0e11, 1.5e11),   bins=50, color='green',  histtype='step', lw=2, label='End Ramp')
ax_bright_b1.hist(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_1']['he_before_SB']['at_end']['brightness'])),       range=(0.0e11, 1.5e11),   bins=50, color='red',    histtype='step', lw=2, label='Start SB')
ax_bright_b1.set_ylabel('Entries', fontweight='bold', fontsize=8)
ax_bright_b1.set_xlabel('B1 Brightness [p/$\mathbf{\mu}$m]', fontweight='bold', fontsize=8)
ax_bright_b1.text(0.6, 1.1, "Injected: {:.2e}$\pm${:.2e} | Start Ramp: {:.2e}$\pm${:.2e} | End Ramp: {:.2e}$\pm${:.2e} | Start SB: {:.2e}$\pm${:.2e}".format(
    np.nanmean(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_1']['Injection']['at_start']['brightness']))), 
    np.nanstd(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_1']['Injection']['at_start']['brightness']))),
    np.nanmean(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_1']['Injection']['at_end']['brightness']))),  
    np.nanstd(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_1']['Injection']['at_end']['brightness']))),
    np.nanmean(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_1']['he_before_SB']['at_start']['brightness']))), 
    np.nanstd(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_1']['he_before_SB']['at_start']['brightness']))),
    np.nanmean(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_1']['he_before_SB']['at_end']['brightness']))),   
    np.nanstd(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_1']['he_before_SB']['at_end']['brightness'])))), 
    horizontalalignment='center', verticalalignment='center', transform=ax_bright_b1.transAxes,
    bbox=dict(facecolor='white', edgecolor='gray', boxstyle='round,pad=0.5', alpha=0.7), fontweight='bold', fontsize=7)
box = ax_bright_b1.get_position()
ax_bright_b1.set_position([box.x0, box.y0, box.width*0.9, box.height])
ax_bright_b1.legend(loc='center left', bbox_to_anchor=(1, 0.5), fontsize=12)
ax_bright_b1.grid('on', which='both')


ax_bright_b2 = pl.subplot(4,2,8)
ax_bright_b2.hist(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_2']['Injection']['at_start']['brightness'])),    range=(0.0e11, 1.5e11),   bins=50, color='blue',   histtype='step',lw=2, label='Injected')
ax_bright_b2.hist(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_2']['Injection']['at_end']['brightness'])),      range=(0.0e11, 1.5e11),   bins=50, color='orange', histtype='step',lw=2, label='Start Ramp')
ax_bright_b2.hist(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_2']['he_before_SB']['at_start']['brightness'])),     range=(0.0e11, 1.5e11),   bins=50, color='green',  histtype='step',lw=2, label='End Ramp')
ax_bright_b2.hist(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_2']['he_before_SB']['at_end']['brightness'])),       range=(0.0e11, 1.5e11),   bins=50, color='red',    histtype='step',lw=2, label='Start SB')
ax_bright_b2.set_ylabel('Entries', fontweight='bold', fontsize=8)
ax_bright_b2.set_xlabel('B2 Brightness [p/$\mathbf{\mu}$m]', fontweight='bold', fontsize=8)
ax_bright_b2.text(0.6, 1.1, "Injected: {:.2e}$\pm${:.2e} | Start Ramp: {:.2e}$\pm${:.2e} | End Ramp: {:.2e}$\pm${:.2e} | Start SB: {:.2e}$\pm${:.2e}".format(
    np.nanmean(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_2']['Injection']['at_start']['brightness']))), 
    np.nanstd(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_2']['Injection']['at_start']['brightness']))),
    np.nanmean(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_2']['Injection']['at_end']['brightness']))),  
    np.nanstd(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_2']['Injection']['at_end']['brightness']))),
    np.nanmean(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_1']['he_before_SB']['at_start']['brightness']))), 
    np.nanstd(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_1']['he_before_SB']['at_start']['brightness']))),
    np.nanmean(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_2']['he_before_SB']['at_end']['brightness']))),   
    np.nanstd(ma.masked_invalid(np.array(dict_intervals_two_beams['beam_2']['he_before_SB']['at_end']['brightness'])))), 
    horizontalalignment='center', verticalalignment='center', transform=ax_bright_b2.transAxes,
    bbox=dict(facecolor='white', edgecolor='gray', boxstyle='round,pad=0.5', alpha=0.7), fontweight='bold', fontsize=7)

box = ax_bright_b2.get_position()
ax_bright_b2.set_position([box.x0, box.y0, box.width*0.9, box.height])

ax_bright_b2.legend(loc='center left', bbox_to_anchor=(1, 0.5), fontsize=12)
ax_bright_b2.grid('on', which='both')
pl.subplots_adjust(left=0.1, wspace=0.5, hspace=0.7)

tref_string = datetime.fromtimestamp(t_ref)
subtitle    = 'Fill {} : Started on {}'.format(filln, tref_string)
fig_hist.suptitle(subtitle, fontsize=16, fontweight='bold')

_____

## CYCLE MODEL PLOTS
---

### Cycle Model Emittance

In [ ]:
t_start_fill, t_end_fill, t_fill_len, t_ref = getCycleDataTimes(bmodes, filln)

t_fill_len = t_end_fill - t_start_fill
t_ref = t_start_fill
t_min = 5.0*60.0

fig_ind = 0
for interval, tag in zip(['Injection', 'he_before_SB'], ["INJ", "FT"]):
    for beam_n in [1,2]:
        # Emittance plot
        fig_emit = pl.figure('emittances_{}_{}'.format(tag, fig_ind), figsize=(15,7))
        Delta_t = (np.array(filln_CycleDict['beam_{}'.format(beam_n)][interval]['at_end']['time_meas'])-np.array(filln_CycleDict['beam_{}'.format(beam_n)][interval]['at_start']['time_meas']))
        filled_slots = filln_CycleDict['beam_{}'.format(beam_n)][interval]['filled_slots']
        mask_valid_bunches = Delta_t>t_min
        
        Delta_emit_meas_H   = (np.array(filln_CycleDict['beam_{}'.format(beam_n)][interval]['at_end']['emith'])-np.array(filln_CycleDict['beam_{}'.format(beam_n)][interval]['at_start']['emith']))
        growth_rate_meas_H  = Delta_emit_meas_H/Delta_t
        Delta_emit_model_H  = (np.array(filln_CycleModelDict['beam_{}'.format(beam_n)][interval]['at_end']['emith'])-np.array(filln_CycleModelDict['beam_{}'.format(beam_n)][interval]['at_start']['emith']))
        growth_rate_model_H = Delta_emit_model_H/Delta_t
        
        Delta_emit_meas_V   = (np.array(filln_CycleDict['beam_{}'.format(beam_n)][interval]['at_end']['emitv'])-np.array(filln_CycleDict['beam_{}'.format(beam_n)][interval]['at_start']['emitv']))
        growth_rate_meas_V  = Delta_emit_meas_V/Delta_t
        Delta_emit_model_V  = (np.array(filln_CycleModelDict['beam_{}'.format(beam_n)][interval]['at_end']['emitv'])-np.array(filln_CycleModelDict['beam_{}'.format(beam_n)][interval]['at_start']['emitv']))
        growth_rate_model_V = Delta_emit_model_V/Delta_t
        
        ax_em_h = fig_emit.add_subplot(211)
        ax_em_h.plot(filled_slots[mask_valid_bunches], 3600.*growth_rate_meas_H[mask_valid_bunches], 'bo', markersize=6, label='Measured')
        ax_em_h.plot(filled_slots[mask_valid_bunches], 3600.*growth_rate_model_H[mask_valid_bunches],'gx', markersize=6 , label='Model')
        ax_em_h.set_ylim(-1, 3)
        ax_em_h.set_ylabel("B{} {}".format(beam_n, tag)+" $\mathbf{\epsilon_{H}}$ Growth [$\mathbf{\mu}$m/h]", fontsize=14, fontweight='bold')
        ax_em_h.grid('on')
        ax_em_h.legend(loc='best', numpoints=1)
        
        ax_em_v = fig_emit.add_subplot(212)
        ax_em_v.plot(filled_slots[mask_valid_bunches], 3600.*growth_rate_meas_V[mask_valid_bunches], 'bo', markersize=6,  label='Measured')
        ax_em_v.plot(filled_slots[mask_valid_bunches], 3600.*growth_rate_model_V[mask_valid_bunches],'gx', markersize=6 , label='Model')
        ax_em_v.set_ylim(-1, 3)
        ax_em_v.set_xlabel('Bunch Slot [25ns]', fontsize=14, fontweight='bold')
        ax_em_v.set_ylabel('B{} {}'.format(beam_n, tag)+' $\mathbf{\epsilon_{V}}$ Growth [$\mathbf{\mu}$m/h]', fontsize=14, fontweight='bold')
        ax_em_v.grid('on')
        # tref string
        tref_string = datetime.fromtimestamp(t_ref)
        subtitle    = 'Fill {} : Started on {}'.format(filln, tref_string)
        fig_emit.suptitle(subtitle, fontsize=16, fontweight='bold')
        ax_em_v.legend(loc='best', numpoints=1)
        fig_ind = fig_ind+1

### Cycle Model Bunch Length

In [ ]:
t_start_fill, t_end_fill, t_fill_len, t_ref = getCycleDataTimes(bmodes, filln)

t_fill_len = t_end_fill - t_start_fill
t_ref = t_start_fill
t_min = 10.0*60.0


i_fig = 0
for interval, tag in zip(['Injection', 'he_before_SB'], ["INJ", "FT"]):

    fig_blen = pl.figure('bucnh length_{}'.format(i_fig), figsize=(15,7))

    Delta_t_b1 = (np.array(filln_CycleDict['beam_1'][interval]['at_end']['time_meas'])-np.array(filln_CycleDict['beam_1'][interval]['at_start']['time_meas']))
    filled_slots_b1 = filln_CycleDict['beam_1'][interval]['filled_slots']
    mask_valid_bunches_b1 = Delta_t_b1>t_min

    Delta_blength_meas_b1   = (np.array(filln_CycleDict['beam_1'][interval]['at_end']['blength'])-np.array(filln_CycleDict['beam_1'][interval]['at_start']['blength']))
    growth_bl_rate_meas_b1  = Delta_blength_meas_b1/Delta_t_b1

    Delta_blength_model_b1  = (np.array(filln_CycleModelDict['beam_1'][interval]['at_end']['blength'])-np.array(filln_CycleModelDict['beam_1'][interval]['at_start']['blength']))
    growth_bl_rate_model_b1 = Delta_blength_model_b1/Delta_t_b1

    Delta_t_b2 = (np.array(filln_CycleDict['beam_2'][interval]['at_end']['time_meas'])-np.array(filln_CycleDict['beam_2'][interval]['at_start']['time_meas']))
    filled_slots_b2 = filln_CycleDict['beam_2'][interval]['filled_slots']
    mask_valid_bunches_b2 = Delta_t_b2>t_min

    Delta_blength_meas_b2   = (np.array(filln_CycleDict['beam_2'][interval]['at_end']['blength'])-np.array(filln_CycleDict['beam_2'][interval]['at_start']['blength']))
    growth_bl_rate_meas_b2  = Delta_blength_meas_b2/Delta_t_b2

    Delta_blength_model_b2  = (np.array(filln_CycleModelDict['beam_2'][interval]['at_end']['blength'])-np.array(filln_CycleModelDict['beam_2'][interval]['at_start']['blength']))
    growth_bl_rate_model_b2 = Delta_blength_model_b2/Delta_t_b2

    ax_blen_b1 = fig_blen.add_subplot(211)
    ax_blen_b1.plot(filled_slots_b1[mask_valid_bunches_b1], 1.0e12*3600.*growth_bl_rate_meas_b1[mask_valid_bunches_b1],  'bo', markersize=6,  label='Measured')
    ax_blen_b1.plot(filled_slots_b1[mask_valid_bunches_b1], 1.0e12*3600.*growth_bl_rate_model_b1[mask_valid_bunches_b1], 'gx', markersize=6 , label="Model")
    ax_blen_b1.set_ylabel('B1 {}'.format(tag)+' Bunch Length Slope [ps/h]'.format(tag=tag), fontsize=10, fontweight='bold')
    ax_blen_b1.grid('on')
    ax_blen_b1.legend(loc='best', numpoints=1)

    ax_blen_b2 = fig_blen.add_subplot(212)
    ax_blen_b2.plot(filled_slots_b2[mask_valid_bunches_b2], 1.0e12*3600.*growth_bl_rate_meas_b2[mask_valid_bunches_b2], 'bo', markersize=6, label='Measured')
    ax_blen_b2.plot(filled_slots_b2[mask_valid_bunches_b2], 1.0e12*3600.*growth_bl_rate_model_b2[mask_valid_bunches_b2],'gx', markersize=6 , label="Model")
    ax_blen_b2.set_xlabel('Bunch Slot [25ns]', fontsize=14, fontweight='bold')
    ax_blen_b2.set_ylabel('B2 '.format(tag)+' Bunch Length Slope [ps/h]', fontsize=10, fontweight='bold')
    ax_blen_b2.grid('on')
    ax_blen_b2.legend(loc='best', numpoints=1)
    # tref string
    tref_string = datetime.fromtimestamp(t_ref)
    subtitle    = 'Fill {} : Started on {}'.format(filln, tref_string)
    fig_blen.suptitle(subtitle, fontsize=16, fontweight='bold')
    i_fig = i_fig+1

_____

## STABLE BEAMS PLOTS
---

### Stable Beams Crossing Angle

In [ ]:
t_start_STABLE, t_end_STABLE, time_range, N_steps = getSBDataTimes(bmodes, filln)
fig_xing = pl.figure(0, figsize=(15, 7))
fig_xing.canvas.set_window_title('CrossingAngle')
fig_xing.set_facecolor('w')
ax = pl.subplot(111)
ax.plot(convertToLocalTime((time_range).astype(int)), 1.0e6*filln_StableBeamsDict['xing_angle'][1]/2.0, 'b', linestyle='-', drawstyle='steps', label='IP1', lw=3)
ax.plot(convertToLocalTime((time_range).astype(int)), 1.0e6*filln_StableBeamsDict['xing_angle'][5]/2.0, 'r', linestyle=':', drawstyle='steps', label='IP5', lw=2)
ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
ax.set_xlabel('Time', fontsize=14, fontweight='bold')
ax.set_ylabel('Half Crossing Angle [$\mathbf{\mu}$rad]', fontsize=14, fontweight='bold')
ax.grid('on')
ax.legend(loc='best', fontsize=12)
tref_string = time.strftime("%a, %d %b %Y %H:%M:%S", time.localtime(t_start_STABLE))
fig_xing.suptitle('Fill {}: STABLE BEAMS declared on {}'.format(filln, tref_string), fontsize=18)


### Stable Beams Emittances

In [7]:
t_start_STABLE, t_end_STABLE, time_range, N_steps = getSBDataTimes(bmodes, filln)

eh_interp_coll               = filln_StableBeamsDict['eh_interp_coll']
ev_interp_coll               = filln_StableBeamsDict['ev_interp_coll']
eh_interp_raw_coll           = filln_StableBeamsDict['eh_interp_raw_coll']
ev_interp_raw_coll           = filln_StableBeamsDict['ev_interp_raw_coll']
slots_filled_coll            = filln_StableBeamsDict['slots_filled_coll']
eh_interp_noncoll            = filln_StableBeamsDict['eh_interp_noncoll']
ev_interp_noncoll            = filln_StableBeamsDict['ev_interp_noncoll']
eh_interp_raw_noncoll        = filln_StableBeamsDict['eh_interp_raw_noncoll']
ev_interp_raw_noncoll        = filln_StableBeamsDict['ev_interp_raw_noncoll']
slots_filled_noncoll         = filln_StableBeamsDict['slots_filled_noncoll']
time_range                   = filln_StableBeamsDict['time_range']

## Figure : Emittances B1
fig_em1 = pl.figure(1, figsize=(15,7))
fig_em1.canvas.set_window_title('Emittances B1')
fig_em1.set_facecolor('w')
ax_ne1h = pl.subplot(2,3,(2,3))
ax_ne1h_t = pl.subplot(2,3,1, sharey=ax_ne1h)
ax_share = ax_ne1h
ax_share_t = ax_ne1h_t
axy_share = ax_ne1h
ax_ne1v = pl.subplot(2,3,(5,6), sharex=ax_share, sharey=axy_share)
ax_ne1v_t = pl.subplot(2,3,4, sharex=ax_share_t, sharey=axy_share)
fig_em1.subplots_adjust(wspace=0.5, hspace=0.5)


## Figure : Emittances B2
fig_em2 = pl.figure(2, figsize=(15,7))
fig_em2.canvas.set_window_title('Emittances B2')
fig_em2.set_facecolor('w')
ax_ne2h = pl.subplot(2,3,(2,3), sharex=ax_share, sharey=axy_share)
ax_ne2v = pl.subplot(2,3,(5,6), sharex=ax_share, sharey=axy_share)
ax_ne2h_t = pl.subplot(2,3,1, sharex=ax_share_t, sharey=axy_share)
ax_ne2v_t = pl.subplot(2,3,4, sharex=ax_share_t, sharey=axy_share)
fig_em2.subplots_adjust(wspace=0.5, hspace=0.5)

## Figure : Emittances B1 Raw
fig_em1_raw = pl.figure(101, figsize=(15,7))
fig_em1_raw.canvas.set_window_title('Emittances B1 raw')
fig_em1_raw.set_facecolor('w')
ax_ne1h_raw = pl.subplot(2,3,(2,3), sharex=ax_share, sharey=axy_share)
ax_ne1h_t_raw = pl.subplot(2,3,1, sharex=ax_share_t, sharey=axy_share)
ax_ne1v_raw = pl.subplot(2,3,(5,6), sharex=ax_share, sharey=axy_share)
ax_ne1v_t_raw = pl.subplot(2,3,4, sharex=ax_share_t, sharey=axy_share)
fig_em1_raw.subplots_adjust(wspace=0.5, hspace=0.5)


## Figure : Emittances B2 Raw
fig_em2_raw = pl.figure(102, figsize=(15,7))
fig_em2_raw.canvas.set_window_title('Emittances B2 raw')
fig_em2_raw.set_facecolor('w')
ax_ne2h_raw = pl.subplot(2,3,(2,3), sharex=ax_share, sharey=axy_share)
ax_ne2v_raw = pl.subplot(2,3,(5,6), sharex=ax_share, sharey=axy_share)
ax_ne2h_t_raw = pl.subplot(2,3,1, sharex=ax_share_t, sharey=axy_share)
ax_ne2v_t_raw = pl.subplot(2,3,4, sharex=ax_share_t, sharey=axy_share)
fig_em2_raw.subplots_adjust(wspace=0.5, hspace=0.5)



plot_mean_and_spread(ax_ne1h_t_raw, convertToLocalTime((time_range).astype(int)), eh_interp_raw_noncoll[1], color='grey', alpha=.5)
plot_mean_and_spread(ax_ne1v_t_raw, convertToLocalTime((time_range).astype(int)), ev_interp_raw_noncoll[1], color='grey', alpha=.5)
plot_mean_and_spread(ax_ne2h_t_raw, convertToLocalTime((time_range).astype(int)), eh_interp_raw_noncoll[2], color='grey', alpha=.5)
plot_mean_and_spread(ax_ne2v_t_raw, convertToLocalTime((time_range).astype(int)), ev_interp_raw_noncoll[2], color='grey', alpha=.5)



plot_mean_and_spread(ax_ne1h_t, convertToLocalTime((time_range).astype(int)), eh_interp_noncoll[1], color='grey', alpha=.5)
plot_mean_and_spread(ax_ne1v_t, convertToLocalTime((time_range).astype(int)), ev_interp_noncoll[1], color='grey', alpha=.5)
plot_mean_and_spread(ax_ne2h_t, convertToLocalTime((time_range).astype(int)), eh_interp_noncoll[2], color='grey', alpha=.5)
plot_mean_and_spread(ax_ne2v_t, convertToLocalTime((time_range).astype(int)), ev_interp_noncoll[2], color='grey', alpha=.5)



plot_mean_and_spread(ax_ne1h_t_raw, convertToLocalTime((time_range).astype(int)), eh_interp_raw_coll[1])
plot_mean_and_spread(ax_ne1v_t_raw, convertToLocalTime((time_range).astype(int)), ev_interp_raw_coll[1])
plot_mean_and_spread(ax_ne2h_t_raw, convertToLocalTime((time_range).astype(int)), eh_interp_raw_coll[2])
plot_mean_and_spread(ax_ne2v_t_raw, convertToLocalTime((time_range).astype(int)), ev_interp_raw_coll[2])



plot_mean_and_spread(ax_ne1h_t, convertToLocalTime((time_range).astype(int)), eh_interp_coll[1])
plot_mean_and_spread(ax_ne1v_t, convertToLocalTime((time_range).astype(int)), ev_interp_coll[1])
plot_mean_and_spread(ax_ne2h_t, convertToLocalTime((time_range).astype(int)), eh_interp_coll[2])
plot_mean_and_spread(ax_ne2v_t, convertToLocalTime((time_range).astype(int)), ev_interp_coll[2])





for ax in [ax_ne1h_t_raw, ax_ne1v_t_raw, ax_ne2h_t_raw, ax_ne2v_t_raw, ax_ne1h_t, ax_ne1v_t, ax_ne2h_t, ax_ne2v_t, ax_ne1h_t_raw, ax_ne1v_t_raw, ax_ne2h_t_raw, ax_ne2v_t_raw, ax_ne1h_t, ax_ne1v_t, ax_ne2h_t, ax_ne2v_t]:
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
    pl.setp( ax.xaxis.get_majorticklabels(), rotation=30 )



for i_time in range(N_steps):
    colorcurr = colorprog(i_prog=i_time, Nplots=N_steps)
    ax_ne1h.plot(slots_filled_coll[1], eh_interp_coll[1][i_time, :], '.', color=colorcurr)
    ax_ne1v.plot(slots_filled_coll[1], ev_interp_coll[1][i_time, :], '.', color=colorcurr)
    ax_ne2h.plot(slots_filled_coll[2], eh_interp_coll[2][i_time, :], '.', color=colorcurr)
    ax_ne2v.plot(slots_filled_coll[2], ev_interp_coll[2][i_time, :], '.', color=colorcurr)

    ax_ne1h_raw.plot(slots_filled_coll[1], eh_interp_raw_coll[1][i_time, :], '.', color=colorcurr)
    ax_ne1v_raw.plot(slots_filled_coll[1], ev_interp_raw_coll[1][i_time, :], '.', color=colorcurr)
    ax_ne2h_raw.plot(slots_filled_coll[2], eh_interp_raw_coll[2][i_time, :], '.', color=colorcurr)
    ax_ne2v_raw.plot(slots_filled_coll[2], ev_interp_raw_coll[2][i_time, :], '.', color=colorcurr)

    ax_ne1h_raw.plot(slots_filled_noncoll[1], eh_interp_raw_noncoll[1][i_time, :], 'x', color=colorcurr)
    ax_ne1v_raw.plot(slots_filled_noncoll[1], ev_interp_raw_noncoll[1][i_time, :], 'x', color=colorcurr)
    ax_ne2h_raw.plot(slots_filled_noncoll[2], eh_interp_raw_noncoll[2][i_time, :], 'x', color=colorcurr)
    ax_ne2v_raw.plot(slots_filled_noncoll[2], ev_interp_raw_noncoll[2][i_time, :], 'x', color=colorcurr)

    ax_ne1h.plot(slots_filled_noncoll[1], eh_interp_noncoll[1][i_time, :], 'x', color=colorcurr)
    ax_ne1v.plot(slots_filled_noncoll[1], ev_interp_noncoll[1][i_time, :], 'x', color=colorcurr)
    ax_ne2h.plot(slots_filled_noncoll[2], eh_interp_noncoll[2][i_time, :], 'x', color=colorcurr)
    ax_ne2v.plot(slots_filled_noncoll[2], ev_interp_noncoll[2][i_time, :], 'x', color=colorcurr)

ax_ne1h_t.set_ylim(0, 5)
ax_ne1v_t.set_ylim(0, 5)
ax_ne2h_t.set_ylim(0, 5)
ax_ne2v_t.set_ylim(0, 5)


for sp in [ax_ne1h, ax_ne1v, ax_ne2h, ax_ne2v, ax_ne1h_raw, ax_ne1v_raw, ax_ne2h_raw, ax_ne2v_raw]:
    sp.grid('on')
    sp.minorticks_on()
    sp.set_xlabel("Bunch Slots [25ns]", fontsize=14, fontweight='bold')

for sp in [ax_ne1h_t, ax_ne1v_t, ax_ne2h_t, ax_ne2v_t, ax_ne1h_t_raw, ax_ne1v_t_raw, ax_ne2h_t_raw, ax_ne2v_t_raw]:
    sp.grid('on')
    sp.minorticks_on()
    sp.set_xlabel('Time', fontsize=14, fontweight='bold')

ax_ne1h_t.set_ylabel('Emittance B1H [$\mathbf{\mu}$m]', fontsize=14, fontweight='bold')
ax_ne1v_t.set_ylabel('Emittance B1V [$\mathbf{\mu}$m]', fontsize=14, fontweight='bold')
ax_ne2h_t.set_ylabel('Emittance B2H [$\mathbf{\mu}$m]', fontsize=14, fontweight='bold')
ax_ne2v_t.set_ylabel('Emittance B2V [$\mathbf{\mu}$m]', fontsize=14, fontweight='bold')

ax_ne1h_t_raw.set_ylabel('Emittance B1H [$\mathbf{\mu}$m]', fontsize=14, fontweight='bold')
ax_ne1v_t_raw.set_ylabel('Emittance B1V [$\mathbf{\mu}$m]', fontsize=14, fontweight='bold')
ax_ne2h_t_raw.set_ylabel('Emittance B2H [$\mathbf{\mu}$m]', fontsize=14, fontweight='bold')
ax_ne2v_t_raw.set_ylabel('Emittance B2V [$\mathbf{\mu}$m]', fontsize=14, fontweight='bold')

tref_string = time.strftime("%a, %d %b %Y %H:%M:%S", time.localtime(t_start_STABLE))
for ff in [fig_em1, fig_em2, fig_em1_raw, fig_em2_raw]:
    ff.suptitle('Fill {}: STABLE BEAMS declared on {}'.format(self.filln, tref_string), fontsize=18)

KeyError: 6432

### Stable Beams Intensity

In [ ]:
t_start_STABLE, t_end_STABLE, time_range, N_steps = getSBDataTimes(bmodes, filln)
b_inten_interp_coll          = self.filln_StableBeamsDict['b_inten_interp_coll']
slots_filled_coll            = self.filln_StableBeamsDict['slots_filled_coll']

b_inten_interp_noncoll       = self.filln_StableBeamsDict['b_inten_interp_noncoll']
slots_filled_noncoll         = self.filln_StableBeamsDict['slots_filled_noncoll']
time_range                   = self.filln_StableBeamsDict['time_range']

fig_int = pl.figure(3, figsize=(15,7))
fig_int.canvas.set_window_title('Bunch intensity')
fig_int.set_facecolor('w')
bx_nb1 = pl.subplot(2,3,(2,3))
bx_nb2 = pl.subplot(2,3,(5,6), sharex=bx_nb1)

bx_nb1_t = pl.subplot(2,3,1, sharey=bx_nb1)
bx_nb2_t = pl.subplot(2,3,4, sharex=bx_nb1_t, sharey=bx_nb2)
fig_int.subplots_adjust(wspace=0.5, hspace=0.5)

plot_mean_and_spread(bx_nb1_t, convertToLocalTime(time_range.astype(int)), b_inten_interp_coll[1])
plot_mean_and_spread(bx_nb2_t, convertToLocalTime(time_range.astype(int)), b_inten_interp_coll[2])

plot_mean_and_spread(bx_nb1_t, convertToLocalTime(time_range.astype(int)), b_inten_interp_noncoll[1], color='grey', alpha=.5)
plot_mean_and_spread(bx_nb2_t, convertToLocalTime(time_range.astype(int)), b_inten_interp_noncoll[2], color='grey', alpha=.5)

for ax in [bx_nb1_t, bx_nb2_t]:
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
    pl.setp( ax.xaxis.get_majorticklabels(), rotation=30 )

for i_time in range(N_steps):
    colorcurr = colorprog(i_prog=i_time, Nplots=N_steps)

    bx_nb1.plot(slots_filled_coll[1], b_inten_interp_coll[1][i_time, :], '.', color=colorcurr)
    bx_nb2.plot(slots_filled_coll[2], b_inten_interp_coll[2][i_time, :], '.', color=colorcurr)

    bx_nb1.plot(slots_filled_noncoll[1], b_inten_interp_noncoll[1][i_time, :], 'x', color=colorcurr)
    bx_nb2.plot(slots_filled_noncoll[2], b_inten_interp_noncoll[2][i_time, :], 'x', color=colorcurr)

for sp in [bx_nb1, bx_nb2]:
    sp.grid('on')
    sp.minorticks_on()
    sp.set_xlabel("Bunch Slots [25ns]", fontsize=14, fontweight='bold')

for sp in [bx_nb1_t, bx_nb2_t]:
    sp.grid('on')
    sp.minorticks_on()
    sp.set_xlabel('Time', fontsize=14, fontweight='bold')

bx_nb1_t.set_ylabel('Intensity B1 [p/b]'  , fontsize=14, fontweight='bold')
bx_nb2_t.set_ylabel('Intensity B2 [p/b]'  , fontsize=14, fontweight='bold')

tref_string = time.strftime("%a, %d %b %Y %H:%M:%S", time.localtime(t_start_STABLE))
fig_int.suptitle('Fill {}: STABLE BEAMS declared on {}'.format(filln, tref_string), fontsize=18)


### Stable Beams Bunch Length

In [ ]:
t_start_STABLE, t_end_STABLE, time_range, N_steps = getSBDataTimes(bmodes, filln)

bl_interp_m_coll             = filln_StableBeamsDict['bl_interp_m_coll']
slots_filled_coll            = filln_StableBeamsDict['slots_filled_coll']

bl_interp_m_noncoll          = filln_StableBeamsDict['bl_interp_m_noncoll']
slots_filled_noncoll         = filln_StableBeamsDict['slots_filled_noncoll']
time_range                   = filln_StableBeamsDict['time_range']

fig_bl = pl.figure(4, figsize=(15,7))
fig_bl.canvas.set_window_title('Bunch length')
fig_bl.set_facecolor('w')
bx_bl1 = pl.subplot(2,3,(2,3))
bx_bl2 = pl.subplot(2,3,(5,6), sharex=bx_bl1)
bx_bl1_t = pl.subplot(2,3,1, sharey=bx_bl1)
bx_bl2_t = pl.subplot(2,3,4, sharex=bx_bl1_t, sharey=bx_bl1)
fig_bl.subplots_adjust(wspace=0.5, hspace=0.5)

plot_mean_and_spread(bx_bl1_t, convertToLocalTime(time_range.astype(int)), bl_interp_m_coll[1]*4/clight*1e9)
plot_mean_and_spread(bx_bl2_t, convertToLocalTime(time_range.astype(int)), bl_interp_m_coll[2]*4/clight*1e9)
plot_mean_and_spread(bx_bl1_t, convertToLocalTime(time_range.astype(int)), bl_interp_m_noncoll[1]*4/clight*1e9, color='grey', alpha=.5)
plot_mean_and_spread(bx_bl2_t, convertToLocalTime(time_range.astype(int)), bl_interp_m_noncoll[2]*4/clight*1e9, color='grey', alpha=.5)
for ax in [bx_bl1_t, bx_bl2_t]:
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
    pl.setp( ax.xaxis.get_majorticklabels(), rotation=30 )

for i_time in range(N_steps):
    colorcurr = self.colorprog(i_prog=i_time, Nplots=N_steps)

    bx_bl1.plot(slots_filled_noncoll[1], bl_interp_m_noncoll[1][i_time, :]*4/clight*1e9, 'x', color=colorcurr)
    bx_bl2.plot(slots_filled_noncoll[2], bl_interp_m_noncoll[2][i_time, :]*4/clight*1e9, 'x', color=colorcurr)			
    
    bx_bl1.plot(slots_filled_coll[1], bl_interp_m_coll[1][i_time, :]*4/clight*1e9, '.', color=colorcurr)
    bx_bl2.plot(slots_filled_coll[2], bl_interp_m_coll[2][i_time, :]*4/clight*1e9, '.', color=colorcurr)

for sp in [bx_bl1, bx_bl2]:
    sp.grid('on')
    sp.minorticks_on()
    sp.set_xlabel("Bunch Slots [25ns]", fontsize=14, fontweight='bold')

for sp in [bx_bl1_t, bx_bl2_t]:
    sp.grid('on')
    sp.minorticks_on()
    sp.set_xlabel('Time', fontsize=14, fontweight='bold')
bx_bl1_t.set_ylabel('Bunch length B1 [ns]', fontsize=14, fontweight='bold')
bx_bl2_t.set_ylabel('Bunch length B1 [ns]', fontsize=14, fontweight='bold')

bx_bl1_t.set_ylim(0.8, 1.2)
bx_bl2_t.set_ylim(0.8, 1.2)

tref_string = time.strftime("%a, %d %b %Y %H:%M:%S", time.localtime(t_start_STABLE))
fig_bl.suptitle('Fill {}: STABLE BEAMS declared on {}'.format(filln, tref_string), fontsize=18)


### Stable Beams Calculated Luminosity

In [ ]:
## first get the times for the fill
t_start_STABLE, t_end_STABLE, time_range, N_steps = getSBDataTimes(bmodes, filln)

## load info from calculated lumi dictionary
lumi_bbb_ATLAS_invm2         = filln_LumiCalcDict['ATLAS']['bunch_lumi']
lumi_bbb_CMS_invm2           = filln_LumiCalcDict['CMS']['bunch_lumi']
slots_filled_coll            = filln_StableBeamsDict['slots_filled_coll']
time_range                   = filln_StableBeamsDict['time_range']

fig_lumi_calc = pl.figure(5, figsize=(15,7))
fig_lumi_calc.canvas.set_window_title('Expected bbb lumis')
fig_lumi_calc.set_facecolor('w')
ax_ATLAS_calc = pl.subplot(2,3,(2,3))
ax_CMS_calc = pl.subplot(2,3,(5,6), sharex=ax_ATLAS_calc, sharey=ax_ATLAS_calc)
ax_ATLAS_calc_t = pl.subplot(2,3,1, sharey=ax_ATLAS_calc)
ax_CMS_calc_t = pl.subplot(2,3,4, sharex=ax_ATLAS_calc_t, sharey=ax_ATLAS_calc)
fig_lumi_calc.subplots_adjust(wspace=0.5, hspace=0.5)

for i_time in range(0, N_steps):
    colorcurr = colorprog(i_prog=i_time, Nplots=N_steps)
    ax_ATLAS_calc.plot(slots_filled_coll[1], lumi_bbb_ATLAS_invm2[i_time, :], '.', color=colorcurr)
    ax_CMS_calc.plot(slots_filled_coll[1],   lumi_bbb_CMS_invm2[i_time, :],   '.', color=colorcurr)

plot_mean_and_spread(ax_ATLAS_calc_t, convertToLocalTime(time_range.astype(int)), lumi_bbb_ATLAS_invm2)
plot_mean_and_spread(ax_CMS_calc_t,   convertToLocalTime(time_range.astype(int)), lumi_bbb_CMS_invm2)
for ax in [ax_ATLAS_calc_t, ax_CMS_calc_t]:
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
    pl.setp( ax.xaxis.get_majorticklabels(), rotation=30 )

for sp in [ax_ATLAS_calc, ax_CMS_calc]:
    sp.grid('on')
    sp.minorticks_on()
    sp.set_xlabel("Bunch Slots [25ns]", fontsize=14, fontweight='bold')

for sp in [ax_ATLAS_calc_t, ax_CMS_calc_t]:
    sp.grid('on')
    sp.minorticks_on()
    sp.set_xlabel('Time', fontsize=14, fontweight='bold')

ax_ATLAS_calc_t.set_ylabel('Calc. Luminosity ATLAS [m$\mathbf{^{-2}}$ s$\mathbf{^{-1}}$]', fontsize=12, fontweight='bold')
ax_CMS_calc_t.set_ylabel('Calc. Luminosity CMS [m$\mathbf{^{-2}}$ s$\mathbf{^{-1}}$]', fontsize=12, fontweight='bold')


tref_string = time.strftime("%a, %d %b %Y %H:%M:%S", time.localtime(t_start_STABLE))
fig_lumi_calc.suptitle('Fill {}: STABLE BEAMS declared on {}'.format(filln, tref_string), fontsize=18)


### Stable Beams Measured Luminosity

In [ ]:
t_start_STABLE, t_end_STABLE, time_range, N_steps = self.getSBDataTimes(bmodes, filln)

slots_filled_coll            = filln_StableBeamsDict['slots_filled_coll']
time_range                   = filln_StableBeamsDict['time_range']

fig_lumi_meas   = pl.figure(7, figsize=(15,7))
fig_lumi_meas.canvas.set_window_title('Measured bbb lumi raw')
fig_lumi_meas.set_facecolor('w')

ax_ATLAS_meas   = pl.subplot(2,3,(2,3))
ax_CMS_meas     = pl.subplot(2,3,(5,6), sharex=ax_ATLAS_meas, sharey=ax_ATLAS_meas)
ax_ATLAS_meas_t = pl.subplot(2,3,1, sharey=ax_ATLAS_meas)
ax_CMS_meas_t   = pl.subplot(2,3,4, sharex=ax_ATLAS_meas_t, sharey=ax_ATLAS_meas)
fig_lumi_meas.subplots_adjust(wspace=0.5, hspace=0.5)

plot_mean_and_spread(ax_ATLAS_meas_t, convertToLocalTime(time_range.astype(int)), filln_LumiMeasDict['ATLAS']['bunch_lumi'])
plot_mean_and_spread(ax_CMS_meas_t,   convertToLocalTime(time_range.astype(int)), filln_LumiMeasDict['CMS']['bunch_lumi'])
for ax in [ax_ATLAS_meas_t, ax_CMS_meas_t]:
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
    pl.setp( ax.xaxis.get_majorticklabels(), rotation=30 )

for i_time in range(0, N_steps):
    colorcurr = colorprog(i_prog=i_time, Nplots=N_steps)
    ax_ATLAS_meas.plot(slots_filled_coll[1], filln_LumiMeasDict['ATLAS']['bunch_lumi'][i_time, :], '.', color=colorcurr)
    ax_CMS_meas.plot(slots_filled_coll[1],   filln_LumiMeasDict['CMS']['bunch_lumi'][i_time, :],   '.', color=colorcurr)

    for sp in [ax_ATLAS_meas, ax_CMS_meas]:
        sp.grid('on')
        sp.set_xlabel("Bunch Slots [25ns]", fontsize=14, fontweight='bold')

    for sp in [ax_ATLAS_meas_t, ax_CMS_meas_t]:
        sp.grid('on')
        sp.set_xlabel('Time', fontsize=14, fontweight='bold')
        sp.set_xlim(pd.to_datetime(t_start_STABLE, unit='s', utc=True).tz_convert('Europe/Zurich').tz_localize(None), pd.to_datetime(t_end_STABLE, unit='s', utc=True).tz_convert('Europe/Zurich').tz_localize(None))
        

    ax_ATLAS_meas_t.set_ylabel('Meas. Luminosity ATLAS [m$\mathbf{^{-2}}$ s$\mathbf{^{-1}}$]', fontsize=12, fontweight='bold')
    ax_CMS_meas_t.set_ylabel('Meas. Luminosity CMS [m$\mathbf{^{-2}}$ s$\mathbf{^{-1}}$]', fontsize=12, fontweight='bold')
    ax_CMS_meas.set_xlim(0, 3564)

tref_string = time.strftime("%a, %d %b %Y %H:%M:%S", time.localtime(t_start_STABLE))
fig_lumi_meas.suptitle('Fill {}: STABLE BEAMS declared on {}'.format(filln, tref_string), fontsize=18)


### Stable Beams Total Luminosity

In [ ]:
t_start_STABLE, t_end_STABLE, time_range, N_steps = getSBDataTimes(bmodes, filln)

lumi_bbb_ATLAS_invm2         = filln_LumiCalcDict['ATLAS']['bunch_lumi']
lumi_bbb_CMS_invm2           = filln_LumiCalcDict['CMS']['bunch_lumi']

fig_total = pl.figure(8, figsize=(15,7))
fig_total.canvas.set_window_title('Total Luminosity')
fig_total.set_facecolor('w')
pl.plot((time_range-t_start_STABLE)/3600., 1e-4*np.sum(self.filln_LumiMeasDict['ATLAS']['bunch_lumi'], axis=1),       color='b', linewidth=2., label="$\mathcal{L}^{meas}_{ATLAS}$")
pl.plot((time_range-t_start_STABLE)/3600., 1e-4*np.sum(lumi_bbb_ATLAS_invm2,                           axis=1), '--', color='b', linewidth=2., label="$\mathcal{L}^{calc}_{ATLAS}$")
pl.plot((time_range-t_start_STABLE)/3600., 1e-4*np.sum(self.filln_LumiMeasDict['CMS']['bunch_lumi'],   axis=1),       color='r', linewidth=2., label="$\mathcal{L}^{meas}_{CMS}$")
pl.plot((time_range-t_start_STABLE)/3600., 1e-4*np.sum(lumi_bbb_CMS_invm2,                             axis=1), '--', color='r', linewidth=2., label="$\mathcal{L}^{calc}_{CMS}$")
ax = pl.gca()
ax.legend(loc='best', prop={"size":12})
ax.set_xlim(-.5, None)
ax.set_ylim(0, None)
ax.set_ylabel('Luminosity [cm$\mathbf{^{-2}}$ s$\mathbf{^{-1}}$]', fontweight='bold', fontsize=14)
ax.set_xlabel('Time [h]', fontsize=14, fontweight='bold')
ax.grid('on')

tref_string = time.strftime("%a, %d %b %Y %H:%M:%S", time.localtime(t_start_STABLE))
fig_total.suptitle('Fill {}: STABLE BEAMS declared on {}'.format(filln, tref_string), fontsize=18)


### Stable Beams Intensity Lifetime

In [ ]:
## first get the times for the fill
t_start_STABLE, t_end_STABLE, time_range, N_steps = self.getSBDataTimes(bmodes, filln)
b_inten_interp_coll          = filln_StableBeamsDict['b_inten_interp_coll']

dict_lifetime = filln_LifetimeDict
if len(dict_lifetime.keys()) == 0:
    dict_lifetime = {1:{}, 2: {}}
    dNdt_bbb            = {}
    dt = filln_StableBeamsDict['time_range'][1]-filln_StableBeamsDict['time_range'][0]
    tau_Np_bbb          = {}
    for beam_n in [1,2 ]:
        dNp_bbb                   = -(b_inten_interp_coll[beam_n][:-1,:]) + (b_inten_interp_coll[beam_n][1:,:])
        dNdt_bbb[beam_n]          = (np.abs(dNp_bbb)/dt)
        tau_Np_bbb[beam_n]        = -1/((dNp_bbb/dt)/b_inten_interp_coll[beam_n][:-1,:])
        dict_lifetime[beam_n]['tau_Np_bbb'] = tau_Np_bbb[beam_n]

fig_bbb_tau = pl.figure('bbb_tau', figsize=(15,7))
fig_bbb_tau.set_facecolor('w')
ax_b1_bbbtau = pl.subplot(211)
ax_b2_bbbtau = pl.subplot(212)

self.plot_mean_and_spread(ax_b1_bbbtau, convertToLocalTime(time_range[0:-1].astype(int)), dict_lifetime[1]['tau_Np_bbb']/3600., label='Beam 1 - $\\tau_{N_{p}^{0}}$'+'={:.2f}h'.format(np.mean(dict_lifetime[1]['tau_Np_bbb']/3600., axis=1)[0]), color='b', shade=True)
ax_b1_bbbtau.grid('on')
ax_b1_bbbtau.set_xlabel('Time', fontsize=14, fontweight='bold')
ax_b1_bbbtau.set_ylabel("$\mathbf{\\tau_{N_{p}}}$ [h]", fontsize=14, fontweight='bold')
ax_b1_bbbtau.legend(loc='best')

self.plot_mean_and_spread(ax_b2_bbbtau, convertToLocalTime(time_range[0:-1].astype(int)), dict_lifetime[2]['tau_Np_bbb']/3600., label='Beam 2 - $\\tau_{N_{p}^{0}}$'+'={:.2f}h'.format(np.mean(dict_lifetime[2]['tau_Np_bbb']/3600., axis=1)[0]), color='r', shade=True)
ax_b2_bbbtau.grid('on')
ax_b2_bbbtau.set_xlabel('Time', fontsize=14, fontweight='bold')
ax_b2_bbbtau.set_ylabel("$\mathbf{\\tau_{N_{p}}}$ [h]", fontsize=14, fontweight='bold')
ax_b2_bbbtau.legend(loc='best')
pl.subplots_adjust(hspace=0.5, wspace=0.5)#, hspace=0.7)

for ax in [ax_b1_bbbtau, ax_b2_bbbtau]:
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))

ax_b1_bbbtau.set_ylim(10,60)
ax_b2_bbbtau.set_ylim(10,60)

tref_string = time.strftime("%a, %d %b %Y %H:%M:%S", time.localtime(t_start_STABLE))
fig_bbb_tau.suptitle('Fill {}: STABLE BEAMS declared on {}'.format(self.filln, tref_string), fontsize=18)

### Stable Beams Normalized Losses

In [ ]:
t_start_STABLE, t_end_STABLE, time_range, N_steps = getSBDataTimes(bmodes, filln)

fig_bbblosses = pl.figure('bbb_losses', figsize=(15,7))
fig_bbblosses.set_facecolor('w')
ax_b1 = pl.subplot(211)
ax_b2 = pl.subplot(212)

plot_mean_and_spread(ax_b1, convertToLocalTime(time_range[0:-1].astype(int)), filln_LifetimeDict[1]['losses_dndtL_bbb']*1.0e31, label='Beam 1 - $\sigma_{eff}^{0}$'+'={:.1f}mb'.format(np.mean(filln_LifetimeDict[1]['losses_dndtL_bbb']*1.0e31, axis=1)[0]), color='b', shade=True)
ax_b1.axhline(80, xmin=0, xmax=1, color='black', label='$\sigma_{\mathrm{inel}}$ = 80mb')
ax_b1.grid('on')
ax_b1.set_xlabel('Time', fontsize=14, fontweight='bold')
ax_b1.set_ylabel("$\mathbf{\left(\\frac{dN}{dt}\\right)\slash\mathcal{L}}$ [mb]", fontsize=14, fontweight='bold')
ax_b1.legend(loc='best')

plot_mean_and_spread(ax_b2, convertToLocalTime(time_range[0:-1].astype(int)), filln_LifetimeDict[2]['losses_dndtL_bbb']*1.0e31, label='Beam 2 - $\sigma_{eff}^{0}$'+'={:.1f}mb'.format(np.mean(filln_LifetimeDict[2]['losses_dndtL_bbb']*1.0e31, axis=1)[0]), color='r', shade=True)
ax_b2.axhline(80, xmin=0, xmax=1, color='black', label='$\sigma_{\mathrm{inel}}$ = 80mb')
ax_b2.grid('on')
ax_b2.set_xlabel('Time', fontsize=14, fontweight='bold')
ax_b2.set_ylabel("$\mathbf{\left(\\frac{dN}{dt}\\right)\slash\mathcal{L}}$ [mb]", fontsize=14, fontweight='bold')
ax_b2.legend(loc='best')
pl.subplots_adjust(hspace=0.5)
for ax in [ax_b1, ax_b2]:
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))

ax_b1.set_ylim(60, 200)
ax_b2.set_ylim(60, 200)

tref_string = time.strftime("%a, %d %b %Y %H:%M:%S", time.localtime(t_start_STABLE))
fig_bbblosses.suptitle('Fill {}: STABLE BEAMS declared on {}'.format(filln, tref_string), fontsize=18)

### Stable Beams Luminosity Lifetime

In [ ]:
turn_around_time_h = 3.5  
t_start_STABLE, t_end_STABLE, time_range, N_steps = getSBDataTimes(bmodes, filln)
slots_filled_coll            = filln_StableBeamsDict['slots_filled_coll']
time_range                   = filln_StableBeamsDict['time_range']


fig_tauLumi_tot = pl.figure('tauLumiTotal', figsize=(15,7))
tau_tot_AT  = pl.subplot(211)
tau_tot_AT.plot(slots_filled_coll[1], -1.0*filln_SBFitsDict['ATLAS']['tau_lumi_calc_coll_full']/3600., c='g', marker='o', markersize=4, ls='None', label='ATLAS - Total Calculated')
tau_tot_AT.plot(slots_filled_coll[1], -1.0*filln_SBFitsDict['ATLAS']['tau_lumi_meas_coll_full']/3600., c='k', marker='o', markersize=4, ls='None', label='ATLAS - Total Measured')
tau_tot_AT.grid('on')
tau_tot_AT.set_ylabel('$\mathbf{\\tau_{\mathcal{L}}}$ [h]', fontsize=14, fontweight='bold')
tau_tot_AT.set_xlabel('Bunch Slots [25ns]', fontsize=14, fontweight='bold')

tau_tot_AT.legend(loc='best', numpoints=1)

tau_tot_CMS = pl.subplot(212)
tau_tot_CMS.plot(slots_filled_coll[1], -1.0*filln_SBFitsDict['CMS']['tau_lumi_calc_coll_full']/3600., c='g', marker='o', markersize=4, ls='None',  label='CMS - Total Calculated')
tau_tot_CMS.plot(slots_filled_coll[1], -1.0*filln_SBFitsDict['CMS']['tau_lumi_meas_coll_full']/3600., c='k', marker='o', markersize=4, ls='None',  label='CMS - Total Measured')  
tau_tot_CMS.grid('on')
tau_tot_CMS.set_ylabel('$\mathbf{\\tau_{\mathcal{L}}}$ [h]', fontsize=14, fontweight='bold')
tau_tot_CMS.set_xlabel('Bunch Slots [25ns]', fontsize=14, fontweight='bold')

tau_tot_CMS.legend(loc='best', numpoints=1)
pl.subplots_adjust(hspace=0.5)


# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
mean_tau_atlas_measured   = np.nanmean(-1.0*self.filln_SBFitsDict['ATLAS']['tau_lumi_meas_coll_full']/3600.)
mean_tau_cms_measured     = np.nanmean(-1.0*self.filln_SBFitsDict['CMS']['tau_lumi_meas_coll_full']/3600.)
mean_tau_atlas_calculated =	np.nanmean(-1.0*self.filln_SBFitsDict['ATLAS']['tau_lumi_calc_coll_full']/3600.)
mean_tau_cms_calculated   = np.nanmean(-1.0*self.filln_SBFitsDict['CMS']['tau_lumi_calc_coll_full']/3600.)

average_measured_lifetime_between_ATLAS_CMS   = (mean_tau_atlas_measured+mean_tau_cms_measured)/2.0
average_calculated_lifetime_between_ATLAS_CMS = (mean_tau_atlas_calculated+mean_tau_cms_calculated)/2.0
optimal_fill_length_measured   = average_measured_lifetime_between_ATLAS_CMS*np.log(1.0+np.sqrt(2*turn_around_time_h/average_measured_lifetime_between_ATLAS_CMS) + turn_around_time_h/average_measured_lifetime_between_ATLAS_CMS)
optimal_fill_length_calculated = average_calculated_lifetime_between_ATLAS_CMS*np.log(1.0+np.sqrt(2*turn_around_time_h/average_calculated_lifetime_between_ATLAS_CMS) + turn_around_time_h/average_calculated_lifetime_between_ATLAS_CMS)
optimal_fill_length_ATLAS_measured = mean_tau_atlas_measured*np.log(1.0+np.sqrt(2*turn_around_time_h/mean_tau_atlas_measured) + turn_around_time_h/mean_tau_atlas_measured)
optimal_fill_length_CMS_measured   = mean_tau_cms_measured*np.log(1.0+np.sqrt(2*turn_around_time_h/mean_tau_cms_measured) + turn_around_time_h/mean_tau_cms_measured)

print "#  Fit Total : Mean Measured ATLAS      = {:.2f} h ".format(mean_tau_atlas_measured)
print "#  Fit Total : Mean Measured CMS        = {:.2f} h ".format(mean_tau_cms_measured)
print "#  Fit Total : Mean Calculated ATLAS    = {:.2f} h ".format(mean_tau_atlas_calculated)
print "#  Fit Total : Mean calculated CMS      = {:.2f} h ".format(mean_tau_cms_calculated)
print "#  ------------------------------------------------"
print "#  Turn-around time                     = {:.2f} h ".format(turn_around_time_h)
print "#  ------------------------------------------------"
print "#  Optimal Fill Length - Measured Avg   = {:.2f} h ".format(optimal_fill_length_measured)
print "#  Optimal Fill Length - Calculated Avg = {:.2f} h ".format(optimal_fill_length_calculated)
print "#  ------------------------------------------------"
print "#  Optimal Fill Length - ATLAS Measured = {:.2f} h ".format(optimal_fill_length_ATLAS_measured)
print "#  Optimal Fill Length - CMS Measured   = {:.2f} h ".format(optimal_fill_length_CMS_measured)
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 

fig_tauLumi_fit = pl.figure('tauLumiFit', figsize=(15,7))
tau_fit_AT  = pl.subplot(211)
tau_fit_AT.plot(slots_filled_coll[1], -1.0*filln_SBFitsDict['ATLAS']['tau_lumi_calc_coll']/3600., c='g', marker='o', markersize=4, ls='None', label='ATLAS - {}h Calculated'.format(int(config.t_fit_length/3600.)))
tau_fit_AT.plot(slots_filled_coll[1], -1.0*filln_SBFitsDict['ATLAS']['tau_lumi_meas_coll']/3600., c='k', marker='o', markersize=4, ls='None', label='ATLAS - {}h Measured'.format(int(config.t_fit_length/3600.)))
tau_fit_AT.grid('on')
tau_fit_AT.set_ylabel('$\mathbf{\\tau_{\mathcal{L}}}$ [h]', fontsize=14, fontweight='bold')
tau_fit_AT.set_xlabel('Bunch Slots [25ns]', fontsize=14, fontweight='bold')
tau_fit_AT.legend(loc='best', numpoints=1)

tau_fit_CMS = pl.subplot(212)
tau_fit_CMS.plot(slots_filled_coll[1], -1.0*filln_SBFitsDict['CMS']['tau_lumi_calc_coll']/3600., c='g', marker='o', markersize=4, ls='None', label='CMS - {}h Calculated'.format(int(config.t_fit_length/3600.)))
tau_fit_CMS.plot(slots_filled_coll[1], -1.0*filln_SBFitsDict['CMS']['tau_lumi_meas_coll']/3600., c='k', marker='o', markersize=4, ls='None', label='CMS - {}h Measured'.format(int(config.t_fit_length/3600.)))
tau_fit_CMS.grid('on')
tau_fit_CMS.set_ylabel('$\mathbf{\\tau_{\mathcal{L}}}$ [h]', fontsize=14, fontweight='bold')
tau_fit_CMS.set_xlabel('Bunch Slots [25ns]', fontsize=14, fontweight='bold')

tau_fit_CMS.legend(loc='best', numpoints=1)
pl.subplots_adjust(hspace=0.5, wspace=0.5)

for sp in [tau_tot_AT, tau_tot_CMS, tau_fit_AT, tau_fit_CMS]:
    sp.set_ylim(5, 30)

tref_string = time.strftime("%a, %d %b %Y %H:%M:%S", time.localtime(t_start_STABLE))
fig_tauLumi_tot.suptitle('Fill {}: STABLE BEAMS declared on {}'.format(filln, tref_string), fontsize=18)
fig_tauLumi_fit.suptitle('Fill {}: STABLE BEAMS declared on {}'.format(filln, tref_string), fontsize=18)